# In What Electoral System Would Donald Trump Have Won The 2020 US Election?

## Introduction

The Electoral College system has been a concrete part of the US presidential election since the founding of the country. In this system, the citizens of each state cast their votes for their choice of electors, who will cast their votes for the presidential candidates. The number of electors of each state is the sum of the number of Senates and the number of Representatives. The Electoral College consists of 538 electors in total, and each elector's vote is counted as 1 vote in the general election. In the end, the candidate who wins over the majority - which means 270 votes - wins the election. The system was created out of good intention to balance the power of small states and large states, however, criticism is getting louder after several instances in which the winner of the popular vote did not win the election due to the outcome of the Electoral College vote. This has led to questions about whether the system accurately represents the will of the US citizens and if there might be a better way to elect the President.

As the fact is that Joe Biden won in the 2020 election, the goal of the paper is to find out whether there exists any electoral system which would have allowed Donald Trump to win in the 2020 election based the original votes.

The proposed electoral systems in this paper ALL have a common presumption, that the voting unit (county or state) which will be responsible to cast ALL of its votes to one candidate. This concept, known as the "winner-take-all" principle, is similar to the existing voting system in 48 of the 50 states and Washington, D.C. in the US.

As data analysis shows that Republican voters are disproportionally located in rural areas and small cities, the 4 electoral systems proposed in the paper all put higher focus on the number of voting unit (county or state), instead of the population of voting unit (county or state). Therefore, 3 of 4 electoral systems makes Donald to win in the 2020 election.

### Variables
- Input $X_1$: Raw votes of Joe Biden (the Democratic Party) in counties
- Input $X_2$: Raw votes of Donald Trump (the Republican Party) in counties
- Input $X_3$: Raw total votes in counties
-- $X_1$, $X_2$ and $X_3$ are all vectors, each component $X_{1,i}$, $X_{2,i}$, $X_{3,i}$, where i $\in$ [1,n], represents the raw votes of DEM, the raw votes of REP and the raw total votes in a specific county, n is the count of counties.
- Output $Y_1$: Fabricated Votes of Joe Biden (the Democratic Party) in US
- Output $Y_2$: Fabricated Votes of Donald Trump (the Republican Party) in US

* **Electoral System I - Elect By County**
Each county has same number of votes as its voters, it casts all of its votes on the party (candidate) that won the majority of votes.
According to the Electoral System I,

$$ Y_1 = \sum \limits _{i=1} ^{|Counties|} (if X_{1,i} > $X_1$), X_{3,i}, 0) $$

$$ Y_2 = \sum \limits _{i=1} ^{|Counties|} (if X_{1,i} < X_{2,i}), X_{3,i}, 0) $$

* **Electoral System II - Elect By County**
Each county has one vote, it casts its vote on the party (candidate) that won the majority of votes.**
According to the Electoral System II,

$$ Y_1 = \sum \limits _{i=1} ^{|Counties|} (if X_{1,i} > X_{2,i}), 1, 0) $$

$$ Y_2 = \sum \limits _{i=1} ^{|Counties|} (if X_{1,i} < X_{2,i}), 1, 0) $$

* **Electoral System III - Elect By State**
Each state has same number of votes as its voters, it casts all of its votes on the party (candidate) that won the majority of counties.
According to the Electoral System III,

$$ Y_{1} = \sum \limits _{i=1} ^{|States|} (if (DEMC_{i} > REPC_{i}), TVofState_{i}, 0) $$

$$ Y_{2} = \sum \limits _{i=1} ^{|States|} (if (DEMC_{i} < REPC_{i}), TVofState_{i}, 0) $$

$$ DEMC_{i} = \sum \limits _{j=1} ^{|Counties-in-a-state_i|} (if (X_{1,j} > X_{2,j}), 1, 0) $$

$$ REPC_{i} = \sum \limits _{j=1} ^{|Counties-in-a-state_i|} ((if (X_{1,j} < X_{2,j}), 1, 0) $$

$$ TVofState_{i} = \sum \limits _{j=1} ^{|Counties-in-a-state_i|} X_{3,j} $$

* **Electoral System IV - Elect By State**
Each state has one vote, it casts its vote on the party (candidate) that won the majority of counties.
According to the Electoral System IV,

$$ Y_{1} = \sum \limits _{i=1} ^{|States|} (if (DEMC_{i} > REPC_{i}), 1, 0) $$

$$ Y_{2} = \sum \limits _{i=1} ^{|States|} (if (DEMC_{i} < REPC_{i}), 1, 0) $$

$$ DEMC_{i} = \sum \limits _{j=1} ^{|Counties-in-a-state_i|} (if (X_{1,j} > X_{2,j}), 1, 0) $$

$$ REPC_{i} = \sum \limits _{j=1} ^{|Counties-in-a-state_i|} (if (X_{1,j} < X_{2,j}), 1, 0) $$

## Data Cleaning/Loading

### Datasets used in this article
1. Kaggle: US Election 2020
    The information includes, states, counties, candidates, parties, and their respective votes.
2. SimpleMaps: US cities except for District of Columbia
    The information includes counties' latitude, and longitude.
3. Census Reporter: District of Columbia
    The information includes 8 Wards of District of Columbia for their latitude, and longitude.
    Note: This information was searched manually, and is manually entered onto the second dataset.

### Data Loading

In [1]:
import pandas as pd
import numpy as np; np.random.seed(42)
import seaborn as sns
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from linearmodels.iv import IV2SLS
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
import requests
from bs4 import BeautifulSoup
import re

*Note a wrong data in president_county_candidate.csv: Maine, Glenwood Plt., Donald Trump, corrected based on
https://www.nytimes.com/interactive/2020/11/03/us/elections/results-maine.html

In [2]:
df = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Kaggle/president_county_candidate.csv')
df.head()

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True


In [3]:
df_pc = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Kaggle/president_county.csv')
df_pc.head()

,state,county,current_votes,total_votes,percent
0,Delaware,Kent County,87025,87025,100
1,Delaware,New Castle County,287633,287633,100
2,Delaware,Sussex County,129352,129352,100
3,District of Columbia,Ward 1,41681,41681,100
4,District of Columbia,Ward 2,32881,32881,100


In [4]:
df_ps = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Kaggle/president_state.csv')
df_ps.head()

,state,total_votes
0,Delaware,504010
1,District of Columbia,344356
2,Florida,11067456
3,Georgia,4997716
4,Hawaii,574469


In [5]:
df_voter = pd.read_csv(
    '/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Kaggle/2020 November General Election - Turnout Rates.csv')
df_voter.head()

,State,Source,Official/Unofficial,Total Ballots Counted (Estimate),Vote for Highest Office (President),VEP Turnout Rate,Voting-Eligible Population (VEP),Voting-Age Population (VAP),% Non-citizen,Prison,Probation,Parole,Total Ineligible Felon,Overseas Eligible,State Abv
0,United States,NaN,NaN,"158,835,004",NaN,66.4%,"239,247,182","257,605,088",7.8%,"1,461,074","1,962,811","616,440","3,294,457","4,971,025",NaN
1,Alabama,https://www2.alabamavotes.gov/electionnight/st...,Unofficial,"2,306,587","2,297,295",62.6%,"3,683,055","3,837,540",2.3%,"25,898","50,997","10,266","67,782",NaN,AL
2,Alaska,https://www.elections.alaska.gov/results/20GEN...,NaN,"367,000",NaN,69.8%,"525,568","551,117",3.4%,"4,293","2,074","1,348","6,927",NaN,AK
3,Arizona,https://results.arizona.vote/#/featured/18/0,NaN,"3,400,000",NaN,65.5%,"5,189,000","5,798,473",8.9%,"38,520","76,844","7,536","93,699",NaN,AZ
4,Arkansas,https://results.enr.clarityelections.com/AR/10...,Unofficial,"1,212,030","1,206,697",55.5%,"2,182,375","2,331,171",3.6%,"17,510","36,719","24,698","64,974",NaN,AR


In [6]:
df_trump_vote = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Other/trump_vote_2020.csv')
df_trump_vote.head()

,state,TRUE,FALSE,Grand Total
0,California,709784,5296177,6005961
1,Texas,3382177,2508170,5890347
2,Florida,3070400,2598331,5668731
3,Pennsylvania,1913864,1464399,3378263
4,New York,1324507,1925723,3250230


In [7]:
df_age = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Other/age.csv')
df_age.head()

,NAME,Median Age (years)
0,Alabama,39.8
1,Alaska,35.6
2,Arizona,30.1
3,Arkansas,37.4
4,California,24.7


In [8]:
usa = gpd.read_file('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Other/cb_2021_us_state_20m/cb_2021_us_state_20m.shp')
usa.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,22,01629543,0400000US22,22,LA,Louisiana,00,111915258185,23736382213,"POLYGON ((-94.04305 32.69303, -94.04303 32.797..."
1,02,01785533,0400000US02,02,AK,Alaska,00,1478942847588,245378425142,"MULTIPOLYGON (((179.48132 51.97530, 179.58286 ..."
2,24,01714934,0400000US24,24,MD,Maryland,00,25151992308,6979074857,"MULTIPOLYGON (((-76.04621 38.02553, -76.00734 ..."
3,55,01779806,0400000US55,55,WI,Wisconsin,00,140292321657,29343646672,"MULTIPOLYGON (((-86.93428 45.42115, -86.83575 ..."
4,12,00294478,0400000US12,12,FL,Florida,00,138961722096,45972570361,"MULTIPOLYGON (((-81.81169 24.56874, -81.75127 ..."


In [9]:
df_geo = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Other/uscities.csv')
df_geo.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18680025,10768.0,shape,False,True,America/New_York,1,11229 11228 11226 11225 11224 11222 11221 1122...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,12531334,3267.0,shape,False,True,America/Los_Angeles,1,91367 90291 90293 90292 91316 91311 90035 9003...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8586888,4576.0,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6076316,4945.0,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7935,-96.7667,5910669,1522.0,shape,False,True,America/Chicago,1,75098 75287 75230 75231 75236 75237 75235 7525...,1840019440


Since the geographical coordinates are incomplete with the above dataset, here I will include another dataset from Wikipedia that provides addition coordinates on the counties with missing data. As this data is important for almost all analysis, including those at the early stage of the project, I decided to move this part to the upfront.

In [10]:
web_url = 'https://en.wikipedia.org/wiki/User:Michael_J/County_table'
response = requests.get(web_url)

soup_object = BeautifulSoup(response.content)

data_table = soup_object.find_all('table', 'wikitable sortable')[0]
all_values = data_table.find_all('tr')

col_names = df_geo.columns.tolist()
county_geo = pd.DataFrame(columns=col_names)
ix = 0
for row in all_values[1:]:
    values = row.find_all('td')
    state_id = values[1].text.split("\n")[0]
    county_fips = int(values[2].text.split("\n")[0])
    county_name = values[3].text.split("\n")[0]
    lat = float(values[12].text.split("\n")[0].replace('+', '').replace('°', ''))
    lng = float(values[13].text.split("\n")[0].replace('–', '').replace('°', ''))
    #population = values[5].text.split("\n")[0].astype(int)*1.074

    county_geo.loc[ix] = ['', '', state_id, '', county_fips, county_name, lat, lng,'','', '', '', '', '', '', '', '', ]
    ix += 1

county_geo.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,,,AL,,1001,Autauga,32.536382,86.644490,,,,,,,,,
1,,,AL,,1003,Baldwin,30.659218,87.746067,,,,,,,,,
2,,,AL,,1005,Barbour,31.870670,85.405456,,,,,,,,,
3,,,AL,,1007,Bibb,33.015893,87.127148,,,,,,,,,
4,,,AL,,1009,Blount,33.977448,86.567246,,,,,,,,,


In [11]:
# assuming df1 and df2 are the two data frames you have
merged_df = pd.concat([df_geo, county_geo], ignore_index=True)

# drop duplicates based on state_id and county_name columns
merged_df.drop_duplicates(subset=['state_id', 'county_name'], inplace=True)
merged_df.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18680025,10768.0,shape,False,True,America/New_York,1,11229 11228 11226 11225 11224 11222 11221 1122...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,12531334,3267.0,shape,False,True,America/Los_Angeles,1,91367 90291 90293 90292 91316 91311 90035 9003...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8586888,4576.0,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6076316,4945.0,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7935,-96.7667,5910669,1522.0,shape,False,True,America/Chicago,1,75098 75287 75230 75231 75236 75237 75235 7525...,1840019440


In [12]:
pd.set_option('display.max_columns', None)

### Data Cleaning
#### Electoral System

##### Step 1: Remove the rows of US Election 2020 where the candidate did not win

In [13]:
df['county'] = df['county'].str.replace(' County', '')
df_pc['county'] = df_pc['county'].str.replace(' County', '')

##### Step 2: Match the geographical coordinates with votes

In [14]:
df_copy = df.copy()
df_pc_copy = df_pc.copy()

df_geo_copy = merged_df.copy()
df_geo_copy.drop(['city', 'city_ascii', 'state_id', 'county_fips', 'source', 'military', 'incorporated', 'timezone', 'ranking', 'zips', 'id', 'population', 'density'], axis = 1, inplace = True)
df_geo_copy.rename(columns={"state_name": "state", "county_name": "county"}, inplace = True)
df_geo_copy.drop_duplicates(subset = ['state', 'county'], keep = 'first', inplace = True)
df_geo_pop = df_geo.copy()
df_geo_pop.drop(['city', 'city_ascii', 'state_id', 'county_fips', 'source', 'military', 'incorporated', 'timezone', 'ranking', 'zips', 'id'], axis = 1, inplace = True)
df_geo_pop.rename(columns={"state_name": "state", "county_name": "county"}, inplace = True)
df_geo_pop.drop_duplicates(subset = ['state', 'county'], keep = 'first', inplace = True)

df_county_statistics = pd.merge(df_copy, df_geo_copy, on=['county', 'state'], how = 'left')
df_pc_copy.drop(['total_votes', 'percent'], axis = 1, inplace = True)
df_county = df_county_statistics.copy()
df_county.drop(df_copy[df_copy['won'] == False].index, inplace = True)
df_county.reset_index(inplace=True)
df_county['total_votes'] = df_pc_copy['current_votes']
df_county.drop(['index'], axis = 1, inplace = True)
df_county.head()

,state,county,candidate,party,total_votes,won,lat,lng
0,Delaware,Kent,Joe Biden,DEM,87025,True,39.1610,-75.5202
1,Delaware,New Castle,Joe Biden,DEM,287633,True,39.7415,-75.5416
2,Delaware,Sussex,Donald Trump,REP,129352,True,38.9091,-75.4227
3,District of Columbia,Ward 1,Joe Biden,DEM,41681,True,38.9072,-77.0369
4,District of Columbia,Ward 2,Joe Biden,DEM,32881,True,38.9063,-77.0340


##### Step 3: Calculating data for summary statistics table

In [15]:
df_county_won = df_county.copy()
df_county_won.drop(df_county_won[df_county_won['candidate'] != 'Donald Trump'].index, inplace = True)
df_county_won.drop(['lat', 'lng'], axis = 1, inplace = True)
df_county_won.rename(columns={"total_votes": "county_total_votes"}, inplace = True)
df_county_won = pd.merge(df_county_won, df_copy, on=['county', 'state', 'candidate', 'party', 'won'], how='left')
df_county_won.rename(columns={"total_votes": "won_votes"}, inplace = True)
df_county_won = df_county_won[['state', 'county', 'candidate', 'party', 'won', 'won_votes', 'county_total_votes']]
df_county_won['share of votes in the winning counties'] = \
    df_county_won['won_votes'] / df_county_won['county_total_votes']
df_county_won.head()

,state,county,candidate,party,won,won_votes,county_total_votes,share of votes in the winning counties
0,Delaware,Sussex,Donald Trump,REP,True,71230,129352,0.550668
1,Florida,Baker,Donald Trump,REP,True,11911,14059,0.847215
2,Florida,Bay,Donald Trump,REP,True,66097,93024,0.710537
3,Florida,Bradford,Donald Trump,REP,True,10334,13632,0.758069
4,Florida,Brevard,Donald Trump,REP,True,207883,360764,0.576230


In [16]:
df_county_lost = df_county_statistics.copy()
df_county_lost.drop(df_copy[df_copy['won'] == True].index, inplace = True)
df_county_lost.drop(df_county_lost[df_county_lost['candidate'] != 'Donald Trump'].index, inplace = True)
df_county_lost.drop(['lat', 'lng'], axis = 1, inplace = True)
df_county_lost.reset_index(inplace=True)
df_county_lost.rename(columns={"total_votes": "won_votes"}, inplace = True)
df_county_lost = pd.merge(df_county_lost, df_pc_copy, on = ['state', 'county'], how = 'left')
df_county_lost.rename(columns={"current_votes": "county_total_votes"}, inplace = True)
df_county_lost.drop(['index'], axis = 1, inplace = True)
df_county_lost = df_county_lost[['state', 'county', 'candidate', 'party', 'won', 'won_votes', 'county_total_votes']]
df_county_lost['share of votes in the losing counties'] =\
    df_county_lost['won_votes'] / df_county_lost['county_total_votes']
df_county_lost.head()

,state,county,candidate,party,won,won_votes,county_total_votes,share of votes in the losing counties
0,Delaware,Kent,Donald Trump,REP,False,41009,87025,0.471232
1,Delaware,New Castle,Donald Trump,REP,False,88364,287633,0.307211
2,District of Columbia,Ward 1,Donald Trump,REP,False,1725,41681,0.041386
3,District of Columbia,Ward 2,Donald Trump,REP,False,2918,32881,0.088744
4,District of Columbia,Ward 3,Donald Trump,REP,False,3705,44231,0.083765


For Electoral System I and II, in which the county is the vote casting level, data cleaning is done. They share the same dataframe for plotting purposes.

For Electoral System III and IV, in which the state is the vote casting level, continue the following steps:

##### Step 4: Calculate the number of counties won by each candidate in each state.

In [17]:
df_state = df_copy.groupby(["state", "candidate", "party"]).sum(numeric_only = True)
df_state.rename(columns={"won": "# of counties won"}, inplace = True)
df_state.index = [df_state.index.get_level_values(0),
                  df_state.index.get_level_values(1),
                  df_state.index.get_level_values(2)]
df_state = df_state.reset_index() \
       .sort_values(['state','# of counties won'], ascending=[True,False]) \
       .set_index(['state','party'])
df_state.drop(['total_votes'], axis = 1, inplace = True)
df_state.head()

candidate  # of counties won
state   party                                 
Alabama REP    Donald Trump                 54
        DEM       Joe Biden                 13
        WRI       Write-ins                  0
        LIB    Jo Jorgensen                  0
Alaska  REP    Donald Trump                 20

##### Step 5: Keep the candidate of the highest voting only

In [18]:
df_state1 = df_state.copy()
df_state1.reset_index(inplace=True)
df_state1.drop_duplicates(
  subset = ['state'],
  keep = 'first', inplace = True)
df_state1 = df_state1.reset_index(drop=True)
df_state1.head()

,state,party,candidate,# of counties won
0,Alabama,REP,Donald Trump,54
1,Alaska,REP,Donald Trump,20
2,Arizona,REP,Donald Trump,10
3,Arkansas,REP,Donald Trump,67
4,California,DEM,Joe Biden,35


##### Step 6: Replace all candidates and parties' name by the winner of each state, because now all the votes of a state belongs to the winner

In [19]:
df_state2 = df_county.copy()
df_state2.drop(['candidate'], axis = 1, inplace = True)
df_state2.drop(['party'], axis = 1, inplace = True)
df_state2 = pd.merge(df_state2, df_state1, on = 'state', how = 'left')
df_state2.head()

,state,county,total_votes,won,lat,lng,party,candidate,# of counties won
0,Delaware,Kent,87025,True,39.1610,-75.5202,DEM,Joe Biden,2
1,Delaware,New Castle,287633,True,39.7415,-75.5416,DEM,Joe Biden,2
2,Delaware,Sussex,129352,True,38.9091,-75.4227,DEM,Joe Biden,2
3,District of Columbia,Ward 1,41681,True,38.9072,-77.0369,DEM,Joe Biden,8
4,District of Columbia,Ward 2,32881,True,38.9063,-77.0340,DEM,Joe Biden,8


##### Step 7: Generate geodataframe for US mainland maps

In [20]:
gpd_state = usa[~usa["NAME"].isin(["Alaska", "Hawaii", "Puerto Rico"])]
gpd_state = gpd_state.copy()
gpd_state.sort_values(by=['NAME'], ascending=True, axis=0, inplace =True)
df_state1.rename(columns={"state": "NAME"}, inplace = True)
gpd_state = pd.merge(gpd_state, df_state1, on="NAME", how="left")
gpd_state.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,party,candidate,# of counties won
0,01,01779775,0400000US01,01,AL,Alabama,00,131175477769,4591897964,"POLYGON ((-88.46866 31.89386, -88.46866 31.933...",REP,Donald Trump,54
1,04,01779777,0400000US04,04,AZ,Arizona,00,294363973043,855871553,"POLYGON ((-114.79968 32.59362, -114.80939 32.6...",REP,Donald Trump,10
2,05,00068085,0400000US05,05,AR,Arkansas,00,134660767709,3121950081,"POLYGON ((-94.61792 36.49941, -94.36120 36.499...",REP,Donald Trump,67
3,06,01779778,0400000US06,06,CA,California,00,403671756816,20293573058,"MULTIPOLYGON (((-118.59397 33.46720, -118.4847...",DEM,Joe Biden,35
4,08,01779779,0400000US08,08,CO,Colorado,00,268418796417,1185716938,"POLYGON ((-109.05996 38.49999, -109.05151 39.1...",REP,Donald Trump,40


##### Step 8: Generate dataframe for mapping the electoral systems

In [21]:
df_state3 = df_state2.copy()
df_state3.drop_duplicates(subset = ['state'], keep = 'first', inplace = True)
df_state3.drop(['county', 'total_votes', 'won', 'lat', 'lng', 'candidate', '# of counties won'], axis = 1, inplace = True)
df_state3 = df_state3[~df_state3["state"].isin(["Alaska", "Hawaii"])]
df_ps_copy = df_ps.copy()
df_state_vote = pd.merge(df_state3, df_ps_copy, on='state', how='left')
df_state_vote.rename(columns={"state": "NAME"}, inplace = True)
gpd_state4 = pd.merge(gpd_state, df_state_vote, on=['NAME', 'party'], how='left')
gpd_state4.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,party,candidate,# of counties won,total_votes
0,01,01779775,0400000US01,01,AL,Alabama,00,131175477769,4591897964,"POLYGON ((-88.46866 31.89386, -88.46866 31.933...",REP,Donald Trump,54,2323304
1,04,01779777,0400000US04,04,AZ,Arizona,00,294363973043,855871553,"POLYGON ((-114.79968 32.59362, -114.80939 32.6...",REP,Donald Trump,10,3387326
2,05,00068085,0400000US05,05,AR,Arkansas,00,134660767709,3121950081,"POLYGON ((-94.61792 36.49941, -94.36120 36.499...",REP,Donald Trump,67,1219069
3,06,01779778,0400000US06,06,CA,California,00,403671756816,20293573058,"MULTIPOLYGON (((-118.59397 33.46720, -118.4847...",DEM,Joe Biden,35,17495906
4,08,01779779,0400000US08,08,CO,Colorado,00,268418796417,1185716938,"POLYGON ((-109.05996 38.49999, -109.05151 39.1...",REP,Donald Trump,40,3256953


##### Step 8: Further data cleaning work for Electoral System III and IV on summary statistics table

In [22]:
df_state_statistics = df_state1.copy()
df_state_total = df_state.groupby(level=[0]).sum(numeric_only = True)
df_state_total.rename(columns={"# of counties won": "# of total counties"}, inplace = True)
df_state_statistics = pd.merge(df_state_statistics, df_state_total, on = 'state', how = 'left')
df_state_statistics.head()

KeyError: 'state'

In [ ]:
df_state_won = df_state_statistics.copy()
df_state_won.drop(df_state_won[df_state_won['candidate'] != 'Donald Trump'].index, inplace = True)
df_state_won['share of counties in the winning states'] = \
    df_state_won['# of counties won'] / df_state_won['# of total counties']
df_state_won['won'] = True
df_state_won = df_state_won[['state', 'candidate', 'party', 'won', '# of counties won', '# of total counties', 'share of counties in the winning states']]
df_state_won.head()

In [ ]:
df_state_lost = df_state1.copy()
df_state_lost.drop(df_state_lost[df_state_lost['candidate'] == 'Donald Trump'].index, inplace = True)
df_state_lost.drop(['party', 'candidate', '# of counties won'], axis = 1, inplace = True)
df_state_lost = pd.merge(df_state_lost, df_state, on = ['state'], how = 'left')
df_state_lost.drop(df_state_lost[df_state_lost['candidate'] != 'Donald Trump'].index, inplace = True)
df_state_lost = pd.merge(df_state_lost, df_state_total, on = ['state'], how = 'left')
df_state_lost['share of counties in the losing states'] = \
    df_state_lost['# of counties won'] / df_state_lost['# of total counties']
df_state_lost['won'] = False
df_state_lost['party'] = 'REP'
df_state_lost = df_state_lost[['state', 'candidate', 'party', 'won', '# of counties won', '# of total counties', 'share of counties in the losing states']]
df_state_lost

#### VEP Turnout Rate

In [ ]:
df_voter2 = pd.DataFrame()
df_voter2['NAME'] = df_voter['State']
df_voter2['VEP Turnout Rate (%)'] = df_voter['VEP Turnout Rate']
df_voter2['Log of Voting-Eligible Population (VEP)'] = \
    np.log10(df_voter['Voting-Eligible Population (VEP)'].replace(',', '',regex=True).astype(int))
df_voter2.head()

#### Summary of Trump Vote at State Level

In [ ]:
df_trump_vote_sum = pd.merge(df_trump_vote, df_ps, on = 'state', how = 'left')
df_trump_vote_sum['ratio'] = df_trump_vote_sum['Grand Total'] / df_trump_vote_sum['total_votes']
df_trump_vote_sum.rename(columns={"state": "NAME"}, inplace = True)
df_trump_vote_sum.head()

#### Summary of Trump Vote at County Level

In [ ]:
df_county_plot = df_county_won.copy()
df_county_plot.rename(columns={'share of votes in the winning counties': 'trump_vote_share'}, inplace = True)
df_county_plot1 = df_county_lost.copy()
df_county_plot1.rename(columns={'share of votes in the losing counties': 'trump_vote_share'}, inplace = True)
df_county_plot = pd.concat([df_county_plot, df_county_plot1])
df_county_plot['trump_vote_share'] = df_county_plot['trump_vote_share']*100
df_county_pop = pd.merge(df_county_plot, df_geo_pop, on = ['state', 'county'])
df_county_pop.drop(df_county_pop[df_county_pop['population'] < df_county_pop['county_total_votes']].index, inplace = True)
df_county_pop['effective_voting_rate'] = df_county_pop['county_total_votes'] / df_county_pop['population'] *100
df_county_pop.head()

## Summary Statistics Tables

In [ ]:
df_I_II_statistics = pd.DataFrame({
    "share of votes in the winning counties" : df_county_won['share of votes in the winning counties'].describe(),
    "share of votes in the losing counties" : df_county_lost['share of votes in the losing counties'].describe(),
})
df_I_II_statistics = df_I_II_statistics.round(decimals = 3)
fig=plt. figure (figsize = (8, 0.3))
ax = fig.add_subplot(111)
ax.table(cellText = df_I_II_statistics.values, colLabels = df_I_II_statistics.columns,
         rowLabels = df_I_II_statistics.index, cellLoc = 'center')
ax.set_title ("In Electoral System I and II, Donald Trump Would Have Won")
ax.axis ('off')
plt.show()

The above summary statistics table presented the basic information about Electoral System I and II. Let's revisit the definition of the two electoral systems. For electoral system I, each county has same number of votes as its voters, it casts all of its votes on the party (candidate) that won the majority of votes. For electoral system II, each county has one vote, it casts its vote on the party (candidate) that won the majority of votes. Both electoral systems share a same characteristic, that is the decision-making level is at the county. Therefore, systematically speaking, there exists a great potential for the counties with large population to overstep the ones with less population, which is not beneficial to the Republican policy. The gap between the two 50%-data is about 31%, keep this data in mind, and we will compare it with the next table.

In [ ]:
df_III_IV_statistics = pd.DataFrame({
    "share of counties in the winning states": df_state_won['share of counties in the winning states'].describe(),
    "share of counties in the losing states": df_state_lost['share of counties in the losing states'].describe(),
})
df_III_IV_statistics = df_III_IV_statistics.round(decimals = 3)
fig2 = plt.figure(figsize = (8, 0.3))
ax2 = fig2.add_subplot(111)
ax2.table(cellText = df_III_IV_statistics.values, colLabels = df_III_IV_statistics.columns,
         rowLabels = df_III_IV_statistics.index, cellLoc = 'center')
ax2.set_title("In Electoral System III and IV, Donald Trump Would Have Won")
ax2.axis ('off')
plt.show()

The above summary statistics table presented the basic information about Electoral System III and IV. Let's revisit the definition of the two electoral systems. For electoral system III, each state has same number of votes as its voters, it casts all of its votes on the party (candidate) that won the majority of counties. For electoral system IV, Each state has one vote, it casts its vote on the party (candidate) that won the majority of counties. Both electoral systems here also share a same characteristic, that is the decision-making level is at the state. Therefore, systematically speaking, no matter how many votes the Democrat receives at the large counties, it only counts as 1 vote per county at the state level. As a result, the advantage of Trump being popular with smaller counties is now fully exploited. The gap between the two 50%-data is about 57%, which is almost a double of the 31% from the above, which proves the previous reasoning.

## Plots, Histograms, Figures
### County Level Analysis

#### Effective Voting Rate Versus Population
The voting-eligible population in the US refers to individuals who are eligible to vote in federal elections, such as presidential, congressional, and senatorial elections. To be eligible, a person must be a citizen of the United States, at least 18 years old, and registered to vote in their state of residence. Excluded from this population are non-citizens, individuals under 18 years old, and unregistered individuals. It is worth noting that while all eligible citizens can vote, not all choose to do so. To better understand the voting patterns in US counties, it is important to analyze not only the eligible voter population but also the level of voter participation. This can be done by evaluating the effectiveness of votes cast by introducing a new concept called Effective Voting Rate, which provides a more accurate picture of the level of voter participation within a county. With that said, allow me to introduce the formula of Effective Voting Rate, which refers to the percentage of effective votes (county_total_votes) in the total population within a county.

$$\text{Effective Voting Rate (\%) in a county} = 100*\frac{\text{Effective Votes in a county}}{\text{Population in a county}} $$

In [ ]:
df_county_pop1 = df_county_pop.copy()
df_county_pop1.drop(df_county_pop1[df_county_pop1['population'] >= 10000].index, inplace = True)
df_county_pop2 = df_county_pop.copy()
df_county_pop2.drop(df_county_pop2[df_county_pop2['population'] < 10000].index, inplace = True)
df_county_pop2.drop(df_county_pop2[df_county_pop2['population'] >= 100000].index, inplace = True)
df_county_pop3 = df_county_pop.copy()
df_county_pop3.drop(df_county_pop3[df_county_pop3['population'] < 100000].index, inplace = True)
df_county_pop3.drop(df_county_pop3[df_county_pop3['population'] >= 1000000].index, inplace = True)
df_county_pop4 = df_county_pop.copy()
df_county_pop4.drop(df_county_pop4[df_county_pop4['population'] < 1000000].index, inplace = True)

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(16, 6), sharey=True)
sns.scatterplot(data=df_county_pop1, x="population", y='effective_voting_rate', hue = "won", palette = "coolwarm", ax = ax1)
ax1.set_title("Population < 10,000", fontsize=14)
ax1.set_xlabel("Population", fontsize=12)
sns.scatterplot(data=df_county_pop2, x="population", y='effective_voting_rate', hue = "won", palette = "coolwarm", ax = ax2)
ax2.set_title("10,000 <= Population < 100,000", fontsize=14)
ax2.set_xlabel("Population", fontsize=12)
sns.scatterplot(data=df_county_pop3, x="population", y='effective_voting_rate', hue = "won", palette = "coolwarm", ax = ax3)
ax3.set_title("100,000 <= Population < 1,000,000", fontsize=14)
ax3.set_xlabel("Population", fontsize=12)
sns.scatterplot(data=df_county_pop4, x="population", y='effective_voting_rate', hue = "won", palette = "coolwarm", ax = ax4)
ax4.set_title("Population >= 1,000,000", fontsize=14)
ax4.set_xlabel("Population", fontsize=12)

ax1.set_ylabel("Effective Voting Rate (%)", fontsize=14)

axes = [ax1, ax2, ax3, ax4]
list_df_pop = [df_county_pop1, df_county_pop2, df_county_pop3, df_county_pop4]
for j in range(len(axes)):
    legend_labels = axes[j].get_legend().get_texts()
    for i, label in enumerate(legend_labels):
        if i == 0:
            label.set_text("Trump Lost")
        if i == 1:
            label.set_text("Trump Won")
    axes[j].get_legend().set_title("Status")
    list_df_pop[j] = list_df_pop[j].dropna(subset=['population', 'effective_voting_rate'])
    X = list_df_pop[j]
    X_won = X[X['won'] == True]["population"]
    y_won = X[X['won'] == True]['effective_voting_rate']
    axes[j].plot(np.unique(X_won),
                 np.poly1d(np.polyfit(X_won, y_won, 1))(np.unique(X_won)),
                 color='red')
    X_lost = X[X['won'] == False]["population"]
    y_lost = X[X['won'] == False]['effective_voting_rate']
    axes[j].plot(np.unique(X_lost),
                 np.poly1d(np.polyfit(X_lost, y_lost, 1))(np.unique(X_lost)),
                 color='blue')
plt.suptitle("Effective Voting Rate vs Population on County-Level", fontsize = 16)
plt.show()

The above 4 scatterplots hold population against Effective Voting Rate with regression lines that demonstrate the linear relationship between the two variables. What we can see from the pattern is that in general, the higher the population, the lower the Effective Voting Rate. By differentiating counties where Trump won or lost, we see that there is smaller and smaller divergence between distribution of the red dots and the blue dots as the population of counties increases. Effective Voting Rate is negatively associated with county population in both conditions.

There is one exception that in the first plot where population is under 10,000, the regression line for counties which Trump had lost is moving slightly upward, opposing the general trend. A possible explanation for this is that in those small counties - almost the same meaning as where Biden won - there might have been a higher level of engagement and motivation among voters, leading to a higher Effective Voting Rate. However, this deviation from the overall pattern is not consistent across all small counties, as the regression line for counties where Trump won remains consistent with the general trend.

Another notable observation is that the slope of the regression line for counties where Trump won is steeper than that of counties where Biden won. This suggests that in counties where Trump had stronger support, the negative association between population and Effective Voting Rate is more pronounced. On the other hand, in counties where Biden had stronger support, the negative association is less steep, indicating that these counties may have a higher level of political engagement or more efficient voting systems that help increase voter turnout. Therefore, we could draw a conclusion that as the population gets higher, Trump is suffering more from the declined Effective Voting Rate than Biden.

#### Effective Voting Rate Versus Density
After conducting an analysis on the relationship between the Effective Voting Rate and population, it is natural to believe that density, another crucial population variable, is likely to play a significant role in influencing the Effective Voting Rate. The association between the two factors is worth exploring further, as it may provide valuable insights into the factors that impact voting behavior. With this in mind, it is important to delve deeper into the relationship between density and Effective Voting Rate to gain a more comprehensive understanding of their interplay.

In [ ]:
df_county_density1 = df_county_pop.copy()
df_county_density1.drop(df_county_density1[df_county_density1['density'] >= 1000].index, inplace = True)
df_county_density2 = df_county_pop.copy()
df_county_density2.drop(df_county_density2[df_county_density2['density'] < 1000].index, inplace = True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
sns.scatterplot(data=df_county_density1, x="density", y='effective_voting_rate', hue = "won", palette = "coolwarm", ax = ax1)
sns.scatterplot(data=df_county_density2, x="density", y='effective_voting_rate', hue = "won", palette = "coolwarm", ax = ax2)
ax1.set_ylabel("Effective Voting Rate", fontsize=14)
ax1.set_title("Density < 1,000", fontsize=14)
ax2.set_title("Density >= 1,000", fontsize=14)
ax1.set_xlabel("Density", fontsize=14)
ax2.set_xlabel("Density", fontsize=14)

axes = [ax1, ax2]
list_df_density = [df_county_density1, df_county_density2]
for j in range(len(axes)):
    legend_labels = axes[j].get_legend().get_texts()
    for i, label in enumerate(legend_labels):
        if i == 0:
            label.set_text("Trump Lost")
        if i == 1:
            label.set_text("Trump Won")
    axes[j].get_legend().set_title("Status")
    list_df_density[j] = list_df_density[j].dropna(subset=['density', 'effective_voting_rate'])
    X = list_df_density[j]
    X_won = X[X['won'] == True]["density"]
    y_won = X[X['won'] == True]['effective_voting_rate']
    axes[j].plot(np.unique(X_won),
                 np.poly1d(np.polyfit(X_won, y_won, 1))(np.unique(X_won)),
                 color='red')
    X_lost = X[X['won'] == False]["density"]
    y_lost = X[X['won'] == False]['effective_voting_rate']
    axes[j].plot(np.unique(X_lost),
                 np.poly1d(np.polyfit(X_lost, y_lost, 1))(np.unique(X_lost)),
                 color='blue')


plt.suptitle("Effective Voting Rate vs Population Density on County-Level", fontsize = 16)
plt.show()

Although the regression lines suggest a weak negative correlation between Effective Voting Rate and density, the scatter dots provide evidence that such a relationship is likely to be insignificant. This finding helps to dispel a potential misconception that may have arisen from the previous discussion, which suggested that a negative correlation between population and Effective Voting Rate would also imply a negative relationship between density and Effective Voting Rate.

However, it is worth noting that the regression line for Trump Won lies above the regression line for Trump lost in both subplots by about 10% Effective Voting Rate. The higher participation rate may be an attribute to Trump's winning in those counties, since more eligible voters were willing to effectively cast their votes when the population density was the same. While the scatterplot in the previous section suggests that Trump was not popular in counties with high population density, this does not necessarily mean that he was also not popular in densely populated counties. In other words, Trump may have had an equal if not a higher chance of winning a small but dense county as Biden did. This highlights the importance of effective voter engagement in the US presidential election.

#### Vote Share of Trump Versus County Total Votes
To further explore the relationship between county size and support for Trump, it is worth examining the vote share of Trump against the total number of votes cast in each county. This can help us determine if Trump's popularity is indeed higher in smaller counties and rural regions, as is commonly believed.

$$\text{Vote Share of Trump (\%) in a county} = 100*\frac{\text{Votes for Trump in a county}}{\text{Total Votes in a county}} $$

In [ ]:
plt.figure(figsize = (8,6))
sns.scatterplot(data = df_county_plot, x = "trump_vote_share", y = 'county_total_votes', hue = "won", size = "won_votes", palette = "coolwarm", sizes=(10,100))

legend_labels = plt.gca().get_legend().get_texts()
for i, label in enumerate(legend_labels):
    if i == 0:
        label.set_text("Status")
    if i == 1:
        label.set_text("Trump Lost")
    if i == 2:
        label.set_text("Trump Won")
    if i == 3:
        label.set_text("Votes for Trump")

plt.title("Vote Share of Trump vs Total Votes on County-Level", fontdict={'family': 'sans-serif', 'size': 14})
plt.xlabel("Vote Share of Trump (%)", fontdict={'family': 'sans-serif', 'size': 12, 'weight': 'normal'})
plt.ylabel("Total Vote", fontdict={'family': 'sans-serif', 'size': 12, 'weight': 'normal'})
plt.show()

The scatterplot above provides a visual representation of the relationship between the number of votes cast in a county and the vote share of Donald Trump in the 2020 presidential election. The plot is divided into two triangular clusters of dots, with blue dots representing counties where Trump lost to other candidates (primarily Joe Biden), and red dots representing counties where Trump emerged as the winner.

Upon closer inspection, we can see that the distribution of dots is denser on the bottom of the red triangle, where Trump won the county. This indicates that Trump was more popular in counties with smaller populations, which tend to be more rural and less densely populated. In contrast, the blue triangle, representing counties where Trump lost, has a higher peak above 4 million votes, indicating that Biden had stronger support in more populous areas.

Overall, the scatterplot confirms the widely held belief that Trump is more popular in rural areas and smaller communities. The stark difference between the peak of the red triangle and that of the blue triangle suggests that Trump's support was more evenly spread across the population in smaller counties, while Biden had more concentrated support in larger counties.

#### Vote Share of Trump Versus Latitude
To further analyze the US political landscape, it is worth considering the role of geography. First we will study one of the two geographical coordinates - latitude. By plotting the share of Trump votes against latitude, the map of the United States is reorganized along the East-West direction, with counties that Trump won on the left and those he lost on the right. The northern counties remain in the north, and the southern counties remain in the south. This allows us to observe potential patterns of support for Trump based on latitude.

In [ ]:
df_county_geo = df_county.copy()
df_county_geo.drop(['candidate', 'party', 'won', 'total_votes'], axis = 1, inplace = True)
df_county_geo = pd.merge(df_county_plot, df_county_geo, on = ['state', 'county'], how = 'left')
df_county_geo = df_county_geo[~df_county_geo["state"].isin(["Alaska", "Hawaii"])]
plt.figure(figsize=(12, 12))
sns.scatterplot(data=df_county_geo, x="trump_vote_share",
                y='lat', hue="won", size="county_total_votes",
                palette="coolwarm", sizes=(10, 100))

plt.axhline(y=39, color='blue', linestyle='dashed', label='Trump Share Dive')
plt.text(88, 38.4, '39th Parallel North', rotation=0, va='bottom', ha='left')
plt.legend()

plt.plot([0, 50], [39, 25], color="magenta", linestyle=':')
plt.plot([0, 50], [39, 50], color="magenta", linestyle=':')
plt.plot([90, 90], [27.5, 48], color="magenta", linestyle=':')
plt.plot([50, 90], [48, 48], color="magenta", linestyle=':')
plt.plot([50, 90], [27.5, 27.5], color="magenta", linestyle=':', label='Trend Boundary')
plt.legend()

legend_labels = plt.gca().get_legend().get_texts()
for i, label in enumerate(legend_labels):
    if i == 0:
        label.set_text("Status")
    if i == 1:
        label.set_text("Trump Lost")
    if i == 2:
        label.set_text("Trump Won")
    if i == 3:
        label.set_text("County's Total Vote")

plt.title("Vote Share of Trump vs Latitude", fontdict={'family': 'sans-serif', 'size': 18})
plt.xlabel("Vote Share of Trump (%)", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})
plt.ylabel("Latitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})

plt.show()

Looking at the scatterplot, we can observe that on the red side where Trump had won, the dots are evenly distributed on both sides of the 39th parallel north, which is the horizontal line dividing the plot into two parts. This indicates that if Trump wins a county, the likelihood of it being located at any latitude is random, and the advantage that Trump has over his opponent is also random, regardless of its geographical location. This suggests that Trump's popularity and vote share in these counties are not significantly influenced by geographical factors.

On the other hand, the blue side where Trump had lost shows a distinctive convergence of dots on the left side of the plot. This suggests that for a county where Trump lost with a weak disadvantage, it is more likely to be located in the north or south rather than in the center of the country. This observation suggests that geographical factors may have had some influence on the outcome of the election in these counties.

#### Trump Vote Share Versus Longitude
After analyzing the relationship between Trump's vote share and latitude, it is also essential to examine the variable of longitude. The following graph displays longitude on the x-axis, reorganizing the US map by the North-South direction, with counties that Trump won at the top and those he lost at the bottom. East counties are still on the right side of the plot, and the west counties are still on the left.

In [ ]:
plt.figure(figsize=(16, 12))
sns.scatterplot(data=df_county_geo, x="lng",
                y='trump_vote_share', hue="won", size="county_total_votes",
                palette="coolwarm", sizes=(10, 100))
plt.axvline(x=-122, color='blue', linestyle='dashed', label='Trump Share Dive')
plt.legend()
plt.text(-122, 5, '-122', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-112, color='red', linestyle='dashed', label='Trump Share Peak')
plt.legend()
plt.text(-112, 95, '-112', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-106, color='blue', linestyle='dashed')
plt.text(-106, 5, '-106', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-101, color='red', linestyle='dashed')
plt.text(-101, 95, '-101', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-90, color='blue', linestyle='dashed')
plt.text(-90, 5, '-90', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-84, color='red', linestyle='dashed')
plt.text(-84, 95, '-84', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-77, color='blue', linestyle='dashed')
plt.text(-77, 5, '-77', rotation=-90, va='bottom', ha='left')

plt.plot([-124, -112], [10, 40], color="magenta", linestyle=':', label='Trend Boundary')
plt.legend()
plt.plot([-124, -112], [70, 90], color="magenta", linestyle=':')
plt.plot([-112, -106], [40, 20], color="magenta", linestyle=':')
plt.plot([-112, -106], [90, 85], color="magenta", linestyle=':')
plt.plot([-106, -101], [20, 40], color="magenta", linestyle=':')
plt.plot([-106, -101], [85, 95], color="magenta", linestyle=':')
plt.plot([-101, -90], [40, 15], color="magenta", linestyle=':')
plt.plot([-101, -90], [95, 85], color="magenta", linestyle=':')
plt.plot([-90, -84], [15, 30], color="magenta", linestyle=':')
plt.plot([-90, -84], [85, 95], color="magenta", linestyle=':')
plt.plot([-84, -73], [30, 5], color="magenta", linestyle=':')
plt.plot([-84, -73], [95, 65], color="magenta", linestyle=':')
plt.legend()

legend_labels = plt.gca().get_legend().get_texts()
for i, label in enumerate(legend_labels):
    if i == 0:
        label.set_text("Status")
    if i == 1:
        label.set_text("Trump Lost")
    if i == 2:
        label.set_text("Trump Won")
    if i == 3:
        label.set_text("County's Total Vote")

plt.title("Vote Share of Trump vs Longitude", fontdict={'family': 'sans-serif', 'size': 18})
plt.xlabel("Longitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})
plt.ylabel("Vote Share of Trump (%)", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})

plt.show()

The scatterplot of longitude against the share of Trump votes reveals a very interesting pattern of distribution. As we move from the farthest west (longitude -122) to the east, we observe that the general votes are relatively unfavored for Donald Trump. However, as we move slightly to the east (longitude -112), we see a local peak in the vote share for Trump. Continuing eastward to longitude -106, we see a local dive in the vote share. This pattern continues, and we observe four total dives at longitudes -122, -106, -90, and -77, and three peaks in between them at longitudes -112, -101, and -84.

This wave-like movement of Trump's vote share from west to east is a fascinating phenomenon that may be influenced by several factors, such as population density, regional culture and values, and historical context. However, the exact cause of this pattern is beyond the scope of this analysis and requires further research. It is possible that future studies may shed light on the underlying reasons for this phenomenon and help us better understand the complex interplay between geography and politics in the United States.

#### Geographical Distribution of Trump Share Votes
By combining the latitude and longitude graph, we have a comprehensive map that illustrates the Trump share of votes in the US mainland. The map is divided into areas represented by blue and red lines, where the blue lines represent areas with lower Trump share, and the red lines represent areas with higher Trump share.

In [ ]:

#plot
fig, gax = plt.subplots(1, figsize=(14, 8))
ax = fig.add_subplot()
gpd_state4.plot(ax=ax, edgecolor="black", color="white")
plt.axvline(x=-122, color='blue', linestyle='dashed', label='Trump Share Dive')
plt.legend()
plt.text(-122, 25, '-122', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-112, color='red', linestyle='dashed', label='Trump Share Peak')
plt.legend()
plt.text(-112, 25, '-112', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-106, color='blue', linestyle='dashed')
plt.text(-106, 25, '-106', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-101, color='red', linestyle='dashed')
plt.text(-101, 25, '-101', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-90, color='blue', linestyle='dashed')
plt.text(-90, 25, '-90', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-84, color='red', linestyle='dashed')
plt.text(-84, 25, '-84', rotation=-90, va='bottom', ha='left')
plt.axvline(x=-77, color='blue', linestyle='dashed')
plt.text(-77, 25, '-77', rotation=-90, va='bottom', ha='left')
plt.axhline(y=39, color='blue', linestyle='dashed')
plt.text(-71, 38, '39th Parallel North', rotation=0, va='bottom', ha='left')

for idx, row in gpd_state4.iterrows():
    name = row['STUSPS']
    if name in ['VT', 'NH', 'MA', 'RI', 'CT', 'NJ', 'MD', 'DE', 'DC']:
        coords = row.geometry.centroid.coords[0]
        if name in ['MA', 'NH', 'JE']:
            ax.annotate(text=name, xy=coords, xytext=(coords[0]+3, coords[1]-1),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=2))
        elif name in ['VT']:
            ax.annotate(text=name, xy=coords, xytext=(coords[0]-1, coords[1]+2),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=2))
        elif name in ['NJ']:
            ax.annotate(text=name, xy=coords, xytext=(coords[0]+1.5, coords[1]-0.5),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=2))
        elif name in ['RI']:
            ax.annotate(text=name, xy=coords, xytext=(coords[0]+3, coords[1]-2),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=2))
        elif name in ['CT']:
            ax.annotate(text=name, xy=coords, xytext=(coords[0]+2, coords[1]-2),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=2))
        elif name in ['DE']:
            ax.annotate(text=name, xy=coords, xytext=(coords[0]+2, coords[1]-0.5),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=2))
        elif name in ['DC']:
            ax.annotate(text=name, xy=coords, xytext=(coords[0]+3, coords[1]-2),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=2))
    else:
        ax.annotate(text=name, xy=row.geometry.centroid.coords[0], ha='center', fontsize=14)
gax.axis('off')
plt.title("Vote Share of Trump Distribution Map", fontdict={'family': 'sans-serif', 'size': 18})
plt.xlabel("Latitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})
plt.ylabel("Longitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})
plt.show()

The map shows that the areas with the highest Trump share are mostly located in the central and southern regions of the country, where more small counties are located. These regions include Texas, Oklahoma, Arkansas, Louisiana, Mississippi, Alabama, Tennessee, Kentucky, West Virginia, and parts of Georgia, South Carolina, and North Carolina. On the other hand, the areas with the lowest Trump share are mostly located in the western and northeastern regions of the country. These regions include California, Oregon, Washington, Minnesota, Illinois, Michigan, New York, Vermont, Massachusetts, Connecticut, and parts of New Hampshire, Maine, and Rhode Island.

The map also reveals a pattern of wave-like movement, where the Trump share of votes tends to peak and dive as we move from west to east. The peaks are located in the western part of Texas, eastern Colorado, and western Ohio, while the dives are located in the western part of Washington, eastern Montana, eastern Iowa, and western Pennsylvania.

Overall, the map provides a comprehensive and detailed view of the distribution of Trump share of votes in the US mainland. It highlights the regions where Trump was more popular and the regions where he was less popular, and it reveals a pattern of wave-like movement that may be worth exploring in future studies.

# Project Two
## The Message
The more the electoral system is leaning towards the smaller counties, the more likely Donald Trump is going to win the election.

In [ ]:
df_boxplot1 = pd.DataFrame({
    'share of votes in the winning counties': df_county_won['share of votes in the winning counties'],
})
df_boxplot1['Electoral Systems'] = 'share of votes in the winning counties'
df_boxplot1.rename(columns={'share of votes in the winning counties': 'Value'}, inplace=True)

df_boxplot2 = pd.DataFrame(data=df_county_lost, columns=['share of votes in the losing counties'])
df_boxplot2['Electoral Systems'] = 'share of votes in the losing counties'
df_boxplot2.rename(columns={'share of votes in the losing counties': 'Value'}, inplace=True)

df_boxplot1 = pd.concat([df_boxplot1, df_boxplot2])
df_boxplot1['Group'] = 'I & II: Elect By County'

df_boxplot3 = pd.DataFrame(data=df_state_won, columns=['share of counties in the winning states'])
df_boxplot3['Electoral Systems'] = 'share of counties in the winning states'
df_boxplot3.rename(columns={'share of counties in the winning states': 'Value'}, inplace=True)

df_boxplot4 = pd.DataFrame(data=df_state_lost, columns=['share of counties in the losing states'])
df_boxplot4['Electoral Systems'] = 'share of counties in the losing states'
df_boxplot4.rename(columns={'share of counties in the losing states': 'Value'}, inplace=True)

df_boxplot3 = pd.concat([df_boxplot3, df_boxplot4])
df_boxplot3['Group'] = 'III & IV: Elect By State'

df_boxplot = pd.concat([df_boxplot1, df_boxplot3])

fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=False)

boxprops = dict(linewidth=2)
medianprops = dict(linewidth=2)
whiskerprops = dict(linewidth=2)
capprops = dict(linewidth=2)

for i, (name, group) in enumerate(df_boxplot.groupby('Group')):
    ax = axes[i]
    group.boxplot(by='Electoral Systems', ax=ax, positions=[1, 2], widths=0.3,
                  boxprops=boxprops, medianprops=medianprops,
                  whiskerprops=whiskerprops, capprops=capprops)

    ax.set_xlabel('')

    if i == 0:
        ax.set_xticklabels(['Won Counties', 'Lost Counties'])
        ax.set_ylabel('Vote Share of Trump (%)')
    else:
        ax.set_xticklabels(['Lost States', 'Won States'])

    ax.set_title(name, fontweight='bold')
    ax.tick_params(axis='both', which='major', labelsize=12)

plt.suptitle('Distribution of Vote Share of Trump by Electoral System', fontweight='bold', fontsize=16)
plt.tight_layout()
plt.show()


Recalling the two general types of electoral systems defined at the beginning of this paper:

*Elect By County:* A county casts all of its votes on the party (candidate) that won the majority of votes.

*Elect By State:* A state casts all of its votes on the party (candidate) that won the majority of counties.

According to these rules, if a county is large enough in the first general type, it has the potential to overstep multiple small counties using its large volume of votes. In contrast, in the second general type, since the decision-making level is at the state, and the majority of counties in a state are small counties, Donald Trump's victory would have been dominant.

The two groups of boxplots presented in this paper show a significant difference in the gap between losing and winning districts between the two general types of electoral systems. The first group, which stands for electing by county, shows that the gap between the two median quantiles marked by the color orange is about 0.3. In contrast, the second group, which represents electing by state, shows that the gap between the two median quantiles marked by the color orange is almost about 0.6, doubling the gap in the first group. A larger gap between the two median quantiles indicates a more overwhelming victory by Trump.

Overall, this observation suggests that the electoral system has a significant impact on the gap between losing and winning districts. This highlights the importance of understanding the weighing by the electoral system analyzing election results.

## Maps and Interpretations
To obtain a clearer representation of the US mainland, we removed Hawaii and Alaska from both county-level and state-level dataframes before generating the graphs.

In [ ]:
df_county_mainland = df_county[~df_county["state"].isin(["Alaska", "Hawaii"])]
df_state_mainland = df_state2[~df_state2["state"].isin(["Alaska", "Hawaii"])]

### Electoral System I - Elect By County
The first electoral system we will examine is Electoral System I, where each county has same number of votes as its voters, and it casts all of its votes on the party (candidate) that won the majority of votes. The formulas been used are as below:

$$ \text{Fabricated Votes for Biden} = \sum \limits _{i=1} ^{|Counties|} (if \text{ Votes for Biden in a county} > \text{Votes for Trump in a county}), \text{Total Votes in a county}, 0) $$

$$ \text{Fabricated Votes for Trump} = \sum \limits _{i=1} ^{|Counties|} (if \text{ Votes for Biden in a county} < \text{Votes for Trump in a county}), \text{Total Votes in a county}, 0) $$

In [ ]:
plt.figure(figsize = (14,10))
sns.scatterplot(data = df_county_mainland, x = "lng", y = "lat", hue = "party", size = "total_votes",
                sizes = (2, 200), palette = ['blue', 'red', 'yellow'])
plt.title("Electoral System I", fontdict={'family': 'sans-serif', 'size': 18})
plt.xlabel("Latitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})
plt.ylabel("Longitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})
legend_labels = plt.gca().get_legend().get_texts()
for i, label in enumerate(legend_labels):
    if i == 0:
        label.set_text("Party That Wins")
    if i == 1:
        label.set_text("Democratic")
    if i == 2:
        label.set_text("Republic")
    if i == 3:
        label.set_text("Liberal")
    if i == 4:
        label.set_text("County's Total Votes")
plt.show()

In [ ]:
y1 = df_county.groupby('party')['total_votes'].sum()['DEM']
y2 = df_county.groupby('party')['total_votes'].sum()['REP']
y3 = df_county.groupby('party')['total_votes'].sum()['LIB']
y4 = df_county.groupby('party')['total_votes'].sum()['WRI']
result_1 = [['DEM', 'Joe Biden', y1, max(y1, y2, y3, y4) == y1],
            ['REP', 'Donald Trump', y2, max(y1, y2, y3, y4) == y2],
            ['LIB', 'Jo Jorgensen', y3, max(y1, y2, y3, y4) == y3],
            ['WRI', 'Write-ins', y4, max(y1, y2, y3, y4) == y4]
            ]
df_result_1 = pd.DataFrame(result_1, columns = ['Party', 'Candidate', 'Fabricated Votes', 'Won'])
df_result_1

The Electoral System I assigns each county a number of votes equal to the number of registered voters in the county, and all of these votes are cast for the party (candidate) that received the majority of votes in that county.

According to the table, Joe Biden emerged as the winner with a total of 94,602,377 votes, and he won the election under this system. In the context of the 2020 United States presidential election, Joe Biden won the majority of votes in a number of megacities, which had a significant impact on the outcome of the election. Biden was able to accumulate a large number of electoral votes due to his success in densely populated areas. On the other hand, Donald Trump won a large number of counties but failed to secure enough votes from densely populated areas to win the election.

### Electoral System II - Elect By County
The second electoral system we will examine is Electoral System II, where each county has one vote, it casts its vote on the party (candidate) that won the majority of votes. The formulas been used are as below:

$$ \text{Fabricated Votes for Biden} = \sum \limits _{i=1} ^{|Counties|} (if \text{ Votes for Biden in a county} > \text{Votes for Trump in a county}), 1, 0) $$

$$ \text{Fabricated Votes for Trump} = \sum \limits _{i=1} ^{|Counties|} (if \text{ Votes for Biden in a county} < \text{Votes for Trump in a county}), 1, 0) $$

In [ ]:
plt.figure(figsize = (14,10))
sns.scatterplot(data = df_county_mainland, x = "lng", y = "lat", hue = "party", size = 20, palette = ['blue', 'red', 'yellow'])
plt.title("Electoral System II", fontdict={'family': 'sans-serif', 'size': 18})
plt.xlabel("Latitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})
plt.ylabel("Longitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})
plt.legend(title='Party That Wins', loc='lower right')
legend_labels = plt.gca().get_legend().get_texts()
for i, label in enumerate(legend_labels):
    if i == 0:
        label.set_text("Democratic")
    if i == 1:
        label.set_text("Republic")
    if i == 2:
        label.set_text("Liberal")
    if i == 3:
        label.set_text("Write-ins")
plt.show()

In [ ]:
y1 = df_county['party'].value_counts()['DEM']
y2 = df_county['party'].value_counts()['REP']
y3 = df_county['party'].value_counts()['LIB']
y4 = df_county['party'].value_counts()['WRI']
result_2 = [['DEM', 'Joe Biden', y1, max(y1, y2, y3, y4) == y1],
            ['REP', 'Donald Trump', y2, max(y1, y2, y3, y4) == y2],
            ['LIB', 'Jo Jorgensen', y3, max(y1, y2, y3, y4) == y3],
            ['WRI', 'Write-ins', y4, max(y1, y2, y3, y4) == y4]
            ]
df_result_2 = pd.DataFrame(result_2, columns = ['Party', 'Candidate', 'Fabricated Votes', 'Won'])
df_result_2

Electoral System II assigns each county a single vote, which is cast for the party (candidate) that won the majority of votes in that county. Under this system, Donald Trump emerged as the winner, with a significant lead over Joe Biden. This is in contrast to the outcome under Electoral System I, where Biden was declared the winner. The difference in the results can be attributed to the fact that in Electoral System II, all counties are weighted equally, regardless of their population or number of registered voters. As a result, Trump's popularity in small counties and traditional Republican strongholds gave him an advantage under this system. In addition, the fact that each county is weighted equally means that the influence of larger urban areas, which tend to lean Democratic, is minimized, leading to a more Republican-leaning outcome.

### Electoral System III - Elect By State
The third electoral system we will examine is Electoral System III, where each state has same number of votes as its voters, it casts all of its votes on the party (candidate) that won the majority of counties. The formulas been used are as below:

$$ \text{Fabricated Votes for Biden} = \sum \limits _{i=1} ^{|States|} (if \text{ State Votes for Biden} > \text{State Votes for Trump}, \text{Total Votes of }State_{i}, 0) $$

$$ \text{Fabricated Votes for Trump} = \sum \limits _{i=1} ^{|States|} (if \text{ State Votes for Biden} < \text{State Votes for Trump}, \text{Total Votes of }State_{i}, 0) $$

$$ \text{State Votes for Biden} = \sum \limits _{j=1} ^{|\text{Counties in a }state_i|} (if \text{ Votes for Biden in a county} > \text{Votes for Trump in a county}, 1, 0) $$

$$ \text{State Votes for Trump} = \sum \limits _{j=1} ^{|\text{Counties in a }state_i|} ((if \text{ Votes for Biden in a county} < \text{Votes for Trump in a county}, 1, 0) $$

$$ \text{Total Votes of }State_{i} = \sum \limits _{j=1} ^{|\text{Counties in a }state_i|} \text{Total Votes in a county} $$

In [ ]:
fig, gax= plt.subplots(figsize=(14, 8))

# Define color mapping
colors = {'DEM': 'blue', 'REP': 'red'}

# Define minimum and maximum values for color scale
vmin = gpd_state4['total_votes'].min()
vmax = gpd_state4['total_votes'].max()

# Plot each state with the appropriate color and shade
for index, row in gpd_state4.iterrows():
    color = colors.get(row['party'], 'gray')
    total_votes = row['total_votes']
    shade = (total_votes - vmin) / (vmax - vmin)
    gpd_state4[gpd_state4.index == index].plot(ax=gax, color=color, edgecolor='black', linewidth=1, alpha=shade)

# Add legend
handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in colors.values()]
labels = colors.keys()
gax.legend(handles, labels, loc='lower right')

for idx, row in gpd_state4.iterrows():
    name = row['STUSPS']
    if name in ['VT', 'NH', 'MA', 'RI', 'CT', 'NJ', 'MD', 'DE', 'DC', 'CA']:
        coords = row.geometry.centroid.coords[0]
        if name in ['MA', 'NH', 'JE']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]+3, coords[1]-1),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black'))
        elif name in ['VT']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]-1, coords[1]+2),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black'))
        elif name in ['NJ']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]+1.5, coords[1]-0.5),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black'))
        elif name in ['RI', 'DC']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]+3, coords[1]-2),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black'))
        elif name in ['CT']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]+2, coords[1]-2),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black'))
        elif name in ['DE']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]+2, coords[1]-0.5),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black'))
        elif name in ['CA']:
            gax.annotate(text=name, xy=row.geometry.centroid.coords[0], ha='center', fontsize=14, color = 'white')
    else:
        gax.annotate(text=name, xy=row.geometry.centroid.coords[0], ha='center', fontsize=14)

# Set plot title
plt.title("Electoral System III", fontdict={'family': 'sans-serif', 'size': 18})
plt.xlabel("Latitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})
plt.ylabel("Longitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})

# Add legend
handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in colors.values()]
labels = ['Democratic', 'Republic']
legend = gax.legend(handles, labels, loc='lower right')

# Set legend title
legend.set_title("Party That Wins")

# Show plot
plt.show()


In [ ]:
y1 = df_state2.groupby('party')['total_votes'].sum()['DEM']
y2 = df_state2.groupby('party')['total_votes'].sum()['REP']
try:
    y3 = df_state2.groupby('party')['total_votes'].sum()['LIB']
except KeyError:
    y3 = 0
try:
    y4 = df_state2.groupby('party')['total_votes'].sum()['WRI']
except KeyError:
    y4 = 0
result_3 = [['DEM', 'Joe Biden', y1, max(y1, y2, y3, y4) == y1],
            ['REP', 'Donald Trump', y2, max(y1, y2, y3, y4) == y2],
            ['LIB', 'Jo Jorgensen', y3, max(y1, y2, y3, y4) == y3],
            ['WRI', 'Write-ins', y4, max(y1, y2, y3, y4) == y4]
            ]
df_result_3 = pd.DataFrame(result_3, columns = ['Party', 'Candidate', 'Fabricated Votes', 'Won'])
df_result_3

Under Electoral System III, Donald Trump won this system with a significant lead over Joe Biden. This can be attributed to the fact that Trump won the majority of counties in many states, including those with a higher number of rural counties. In contrast, Biden's support was mainly concentrated in urban areas and some Democrat's conservative base states. The result is notably different from the previous two systems, as the number of small counties is significantly higher than the number of large counties, and that the state-level decision is purely based on which candidate wins the majority counties. This system gives substantial advantage to Donald Trump, due to the popularity of his policy among people live in the small towns and rural regions. In Electoral System III, because the number of counties won by a party (candidate) plays a significant role in determining the winner, while in the previous two systems, the total number of votes won by a party (candidate) was the deciding factor, the system heavily leans towards small counties rather than populated cities.

### Electoral System IV - Elect By State
The last electoral system we will examine is Electoral System IV, where each state has one vote, it casts all of its votes on the party (candidate) that won the majority of counties. The formulas been used are as below:

$$ \text{Fabricated Votes for Biden} = \sum \limits _{i=1} ^{|States|} (if \text{ State Votes for Biden} > \text{State Votes for Trump}, 1, 0) $$

$$ \text{Fabricated Votes for Trump} = \sum \limits _{i=1} ^{|States|} (if \text{ State Votes for Biden} < \text{State Votes for Trump}, 1, 0) $$

$$ \text{State Votes for Biden} = \sum \limits _{j=1} ^{|\text{Counties in a }state_i|} (if \text{ Votes for Biden in a county} > \text{Votes for Trump in a county}, 1, 0) $$

$$ \text{State Votes for Trump} = \sum \limits _{j=1} ^{|\text{Counties in a }state_i|} ((if \text{ Votes for Biden in a county} < \text{Votes for Trump in a county}, 1, 0) $$

$$ \text{Total Votes of }State_{i} = \sum \limits _{j=1} ^{|\text{Counties in a }state_i|} \text{Total Votes in a county} $$

In [ ]:
fig, gax= plt.subplots(1, figsize = (14,8))
gpd_state4.plot(ax=gax, edgecolor='black', column='party', legend=True, cmap='bwr', categorical=True, vmin=-1, vmax=2)
for idx, row in gpd_state4.iterrows():
    name = row['STUSPS']
    if name in ['VT', 'NH', 'MA', 'RI', 'CT', 'NJ', 'MD', 'DE', 'DC']:
        coords = row.geometry.centroid.coords[0]
        if name in ['MA', 'NH', 'JE']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]+3, coords[1]-1),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=1.5))
        elif name in ['VT']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]-1, coords[1]+2),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=1.5))
        elif name in ['NJ']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]+1.5, coords[1]-0.5),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=1.5))
        elif name in ['RI']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]+3, coords[1]-2),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=1.5))
        elif name in ['CT']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]+2, coords[1]-2),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=1.5))
        elif name in ['DE']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]+2, coords[1]-0.5),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=1.5))
        elif name in ['DC']:
            gax.annotate(text=name, xy=coords, xytext=(coords[0]+3, coords[1]-2),
                        ha='center', fontsize=14, arrowprops=dict(arrowstyle="-", color='black', linewidth=1.5))
    else:
        gax.annotate(text=name, xy=row.geometry.centroid.coords[0], ha='center', fontsize=14)
plt.title("Electoral System IV", fontdict={'family': 'sans-serif', 'size': 18})
plt.xlabel("Latitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})
plt.ylabel("Longitude", fontdict={'family': 'sans-serif', 'size': 16, 'weight': 'normal'})
plt.show()

In [ ]:
y1 = df_state1['party'].value_counts()['DEM']
y2 = df_state1['party'].value_counts()['REP']
try:
    y3 = df_state1['party'].value_counts()['LIB']
except KeyError:
    y3 = 0
try:
    y4 = df_state1['party'].value_counts()['WRI']
except KeyError:
    y4 = 0
result_4 = [['DEM', 'Joe Biden', y1, max(y1, y2, y3, y4) == y1],
            ['REP', 'Donald Trump', y2, max(y1, y2, y3, y4) == y2],
            ['LIB', 'Jo Jorgensen', y3, max(y1, y2, y3, y4) == y3],
            ['WRI', 'Write-ins', y4, max(y1, y2, y3, y4) == y4]
            ]
df_result_4 = pd.DataFrame(result_4, columns = ['Party', 'Candidate', 'Fabricated Votes', 'Won'])
df_result_4

### Summary of Four Electoral Systems

In [ ]:
result = df_result_1.copy()
result.drop(['Candidate', 'Won'], axis = 1, inplace = True)
result.rename(columns={"Fabricated Votes": "I"}, inplace = True)
result = result.T
new_header = result.iloc[0]
result = result[1:]
result.columns = new_header

result_add = df_result_2.copy()
result_add.drop(['Candidate', 'Won'], axis = 1, inplace = True)
result_add.rename(columns={"Fabricated Votes": "II"}, inplace = True)
result_add = result_add.T
new_header = result_add.iloc[0]
result_add = result_add[1:]
result_add.columns = new_header
result = pd.concat([result, result_add])

result_add = df_result_3.copy()
result_add.drop(['Candidate', 'Won'], axis = 1, inplace = True)
result_add.rename(columns={"Fabricated Votes": "III"}, inplace = True)
result_add = result_add.T
new_header = result_add.iloc[0]
result_add = result_add[1:]
result_add.columns = new_header
result = pd.concat([result, result_add])

result_add = df_result_4.copy()
result_add.drop(['Candidate', 'Won'], axis = 1, inplace = True)
result_add.rename(columns={"Fabricated Votes": "IV"}, inplace = True)
result_add = result_add.T
new_header = result_add.iloc[0]
result_add = result_add[1:]
result_add.columns = new_header
result = pd.concat([result, result_add])

result

In [ ]:
stacked_data = result.apply(lambda x: x*100/sum(x), axis=1)
stacked_data.plot(kind="bar", stacked=True, color = ['blue', 'red', 'yellow', 'grey'])
plt.title("Final Result of All Four Electoral Systems")
plt.xlabel("Electoral System")
plt.ylabel("Percentage Fabricated Vote of Parties (%)")
plt.show()

As from the above bar graph, the advantage Trump obtained in each of the electoral system is gradually increasing from the first one to the last one. This is because the weight small counties carry is increasing from left to right. It goes back to the message that Trump is favored by the people living in smaller counties, and if such group is given more weight, the likelihood of Trump winning the election will also increase.

# Project Three
## Potential Data to Scrape
One potential addition to the campaign spending data by candidate and state in the 2020 US presidential election dataset could be web scraping data on funding sources by sector. By scraping data on the fundraising, we could gain insights into which sector were spending money to support or oppose the candidates, and by how much.

One potential website to scrape this data from is OpenSecrets.org, which is a website that tracks money in politics. OpenSecrets.org provides data on funding source by candidate and by sector, which could help shed light on which groups were most active in trying to influence the election on a county-level.

To have an understanding of how this data could be parallel on our previous findings, we can take a look at whether the funding from the argricultural sector confirmed that Trump's policy is more appealing to people and businesses in the rural places, so that he attracts more funding than Biden; and whether Biden's policy is more attractive to urban cities so that finance sector is more willing to support him rather than Trump.

The above are just examples on how we could then use this new information to help answer my research question, the discussion later in this chapter will analyze the data in detail.

## Potential Challenges
Firstly, the website does not provide all the data that we need in a single location or in a format that is easy to scrape. We need to scrape data from multiple pages or even multiple websites to gather all the relevant information.

Secondly, merging the scraped data with our existing dataset may require more information on a state-level, which the website does not provide complete information on. In future work, we can find such information either by searching in other websites, or we can contact the website holder OpenSecrets for a more detailed dataset.

Side Note: I have also found a dataset that contains contribution details including industry, organization name, amount and states on https://www.fec.gov/data/candidates/president/presidential-map/. However, the size of the file was too large to be handled, therefore I choose to not use this in Project 3.

Here is the file:

In [ ]:
#df_candidate_funding = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Other/P00000001-ALL.csv')
#df_candidate_funding.head(10)

## Scraping Data from a Website
Since the website I am working on does not have all the information on one page, therefore I have to run a for loop to scrape all the related HTML pages.

Create essential lists that will be used in scrapping. Since the funds can come from two channels, one is campaign committee, the other one is outside groups, we will also set a third channel which combines the previous two.

In [ ]:
sectors = ['Agribusiness', 'Communications/Electronics', 'Construction', 'Defense', 'Energy/Natural Resources', 'Finance/Insurance/Real Estate', 'Health', 'Lawyers & Lobbyists', 'Transportation', 'Misc Business', 'Labor', 'Ideology/Single-Issue', 'Other']

# Total funding from campaign committee and outside groups by sector
channel_combined = ['https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=A',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=B',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=C',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=D',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=E',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=F',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=H',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=K',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=M',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=N',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=P',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=Q',
                'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=W']

# Funding from campaign committee by sector
channel_campaign_committee = ['https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=A&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=B&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=C&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=D&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=E&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=F&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=H&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=K&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=M&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=N&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=P&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=Q&src=c',
                            'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=W&src=c']

# Funding from outside groups by sector
channel_outside_groups = ['https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=A&src=o',
                          'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=B&src=o',
                        'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=C&src=o',
                        'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=D&src=o',
                        'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=E&src=o',
                        'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=F&src=o',
                        'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=H&src=o',
                        'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=K&src=o',
                        'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=M&src=o',
                        'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=N&src=o',
                        'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=P&src=o',
                        'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=Q&src=o',
                        'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=W&src=o']

channel_fund = [channel_combined, channel_campaign_committee, channel_outside_groups]

For explanation purposes, I will use the first website link:'https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=A' to demonstrate detailed steps.


The requests library can be used to work with webpages and web content. A request is made to get the content of the webpage with the get() method.

In [ ]:
import requests
from bs4 import BeautifulSoup

web_url = "https://www.opensecrets.org/2020-presidential-race/sector-totals?sector=A"
response = requests.get(web_url)

A response is received from the web server. This response will have response headers and status codes associated to that particular request. Response headers give the detailed information about the request made to the web server.

In [ ]:
response.headers

Every response will have a status code. The status codes indicate whether a specific HTTP request has been successfully completed. Responses are grouped in five classes:

Informational responses (100–199)
Successful responses (200–299)
Redirects (300–399)
Client errors (400–499)
Server errors (500–599)
Sometimes you get a client error like (403), then you should add your user agent as header to your Get request (google how to get user agent), you can also look at this stackoverflow page: https://stackoverflow.com/questions/38489386/python-requests-403-forbidden

Let us check the response status code for the HTTP request we placed

In [ ]:
response.status_code

The response has a status code of 200. This is a successful response and hence there should be relevant content of the webpage in the obtained response. This can be checked by printing the content. This content received is the HTML source code of the webpage.

### Parsing HTML and accessing different elements
bs4 is a Python library which parses through HTML content and understands the complete structure of the content. The response content can be passed to a BeautifulSoup() method to obtain a soup object which looks very structured.

In [ ]:
soup_object = BeautifulSoup(response.content, 'html.parser')

str(soup_object)[0:1000]

### Explore the schema: Manipulating it into a tabular structure
---
To be able to accurately extract relevant data from the webpage, it is important to explore the schema and understand the structure of the webpage. A good way to do this is to inspect the webpage directly on a web browser.
<br>

To do this,
- Open the webpage on a browser
- Right click on the data content to be extracted
- Click on `Inspect` or `Inspect element` option

This will open a console window which shows the real time HTML code corresponding to the web content. Now identify the type of HTML tag which contains all the data along with any id names or class names associated to that HTML tag. In our case, the data is enclosed in the `<table>` HTML tag with the class name `'data-table'`. This information can be used to search for the web content directly in our soup object with the `find_all()` method. This will return a soup object.

It is important to look for <table class="??????">, where the "??????" will be used down below. In our case, the "??????" would be "DataTable":

...
<table class="DataTable" data-order='[[ 1, "desc" ]]'>
<thead>
<tr>
<th>Candidate</th>
<th class="number">Total</th>
</tr>
</thead>
<tbody>
<tr>
<td><a href="/2020-presidential-race/donald-trump/candidate?id=N00023864">Trump, Donald (R)</a></td>
<td class="number">$16,091,308</td>
</tr>
<tr>
...

In [ ]:
data_table = soup_object.find_all('table', 'DataTable')[0]

str(data_table)[0:1000]

It can be seen that relevant block of data has been extracted but further extraction needs to be done to individually extract the rank, company name and the company website data. On further analysis, it can be seen that every row of data is enclosed under a `<tr>` HTML tag which means `table row`. All these row values can be extracted into a list of values by finding the `<tr>` values from our newly created soup object `data_table`.

`<th>` means header row.
`<td>` defines a standard data cell in an HTML table.
`<a>`  defines a hyperlink.
If you happen to see a tag that you don't know what it means (which is most of the time!), you can google the name of the tage + HTML; for example, google this `<th> HTML`

Take a closer look at the data that we are interested in:

In [ ]:
all_values = data_table.find_all('tr')
all_values[:3] # Prints the first 3 captured tag elements

In [ ]:
print(all_values[0])
print('--')
print(all_values[1])
print('--')
print(all_values[2])

### Organize the value we find
The first element of the list contains the column names 'Candidate' and 'Total'. The next elements of the list contain soup objects which contain the funding data. This data can be extracted in a loop since the structure for all the list elements is the same.

- An empty dataframe `df_fund` is created with the column names 'Candidate' and 'Total'
- The index is initiated to zero
- A for loop is designed to go through all the elements of the list in order and extract the candidate name and total funding amount from the list element which are enclosed in the `<td>` HTML tag. A `find_all()` will return a list of td tags.
- The `.text` attribute can be used to just pick the text part from the tag. In our case this is the rank, company name and the company website
- These values are then put into the dataframe and the index value is incremented

In [ ]:
all_values[1].find_all('td')

In [ ]:
all_values[1].find_all('td')[0].text

In [ ]:
all_values[1].find_all('td')[1].text

In [ ]:
df_fund = pd.DataFrame(columns = ['Candidate', 'Total'])
ix = 0 # Initializing the row index for the DataFrame

# Scraping data from each row in the table and storing it in the DataFrame
for row in all_values[1:]:
    values = row.find_all('td')
    candidate = values[0].text
    total = values[1].text
    df_fund.loc[ix] = [candidate, total]# Store it in the dataframe as a row
    ix += 1

# Print the first 5 rows of the dataframe
df_fund.head()

Now the scrapping demonstration for one website is completed. Let's use a for loop to iterate through all the websites.

In [ ]:
# Part 1: Initializing the Data Structures
df_funds = list() #Creating an empty list to store all sector funds data.

# Part 2: Scraping Data for each channel (Combined, Campaign Committee, Outside Groups)
for fund in channel_fund:
    df_total_lists = [] # Creating an empty list to store each channel's data for all sectors
    for i in range(len(sectors)):
        totals = fund[i] # Generating the url to scrape data from
        response = requests.get(totals) # Requesting the web page for the url
        soup_object = BeautifulSoup(response.content, 'html.parser') # Parsing the html content using BeautifulSoup
        data_table = soup_object.find_all('table', 'DataTable')[0] # Finding the table containing the data we want to scrape
        all_values = data_table.find_all('tr') # Finding all rows in the table
        df_fund = pd.DataFrame(columns = ['Candidate', sectors[i]]) # Creating a DataFrame to store the scraped data for a single sector

        ix = 0 # Initializing the row index for the DataFrame
        # Scraping data from each row in the table and storing it in the DataFrame
        for row in all_values[1:]:
            values = row.find_all('td')
            candidate = values[0].text
            total = values[1].text
            df_fund.loc[ix] = [candidate, total]
            ix += 1

        df_total_lists.append(df_fund) # Storing the DataFrame for a single sector in the list of DataFrames for all sectors

    # Part 3: Merging Channel Data for fund
    # Performing a left join on the 'Candidate' column of all sector DataFrames
    df_channel_combined = df_total_lists[0]
    for df in df_total_lists[1:]:
        df_channel_combined = pd.merge(df_channel_combined, df, on='Candidate', how='left')

    df_funds.append(df_channel_combined)# Storing the merged DataFrame for a single channel in the list of all channel funds data

In [ ]:
import re

# Part 4: Convert DataFrame tp plot-friendly
# Change the datatype from object to integer, and convert the unit into millions.
df_funds_copy = df_funds.copy()
for df in df_funds:
    for col in df.columns[1:]:
        df[col] = df[col].apply(lambda x: int(re.sub(r'\D', '', str(x))) if str(x) != 'nan' else 0)
        df[col] = df[col] / 1000000

2020 Trump Presidential Campaign used for linear regression:

In [ ]:
web_url = 'https://en.wikipedia.org/wiki/List_of_post%E2%80%932016_election_Donald_Trump_rallies'
response = requests.get(web_url)

soup_object = BeautifulSoup(response.content)

Rallies_2020=pd.DataFrame()

for i in [3,4,5]:

    data_table = soup_object.find_all('table', 'wikitable sortable')[i]
    all_values = data_table.find_all('tr')

    Rallies2020 = pd.DataFrame(columns = ['City', 'State', 'Venue', 'Other speakers'])
    ix = 0

    for row in all_values[1:]:
        values = row.find_all('td')

        city = values[0].text.split("\n")[0]
        if len(values[1].text.split("\n")[0]) == 2:
            state = values[1].text.split("\n")[0]
            venue = values[2].text.split("\n")[0]
            other = values[3].text.split("\n")[0]
        else:
            venue = values[1].text.split("\n")[0]
            other = values[2].text.split("\n")[0]
        Rallies2020.loc[ix] = [city, state, venue, other]
        ix += 1

    Rallies_2020 = pd.concat([Rallies_2020, Rallies2020], axis=0)

Rallies_2020.head()#A total of 100 rallies

## Merging the Scraped Dataset
The previous results achieved in Project 1 and 2 are mostly discussing the vote on a geographical level such as county or state. Since I introduced a new dimension - sector - in this section, it is not very suitable for merging the datasets together.

In [ ]:
# Count entries in the first dataframe (total funding for sectors)
df_funds[0].info()

Up to this point, we have three dataframes of the same size, the total # of observations from the webscrapping is 33*14*3 = 1386

## Visualizing the Scraped Dataset

In [ ]:
# Loading dataframes
df0 = df_funds[0].copy()
df1 = df_funds[1].copy()
df2 = df_funds[2].copy()

In [ ]:
# TABLE
# Selecting needed rows from dataframes
trump_cc = pd.DataFrame(df1.iloc[0]).transpose()
trump_og = pd.DataFrame(df2.iloc[0]).transpose()
trump_c = pd.DataFrame(df0.iloc[0]).transpose()
biden_cc = pd.DataFrame(df1.iloc[1]).transpose()
biden_og = pd.DataFrame(df2.iloc[1]).transpose()
biden_c = pd.DataFrame(df0.iloc[1]).transpose()

# Combining all the rows to one dataframe
stacked_df = pd.concat([trump_cc, trump_og, trump_c, biden_cc, biden_og, biden_c], axis=0)
stacked_df['Candidate'] = ['Trump - Campaign Committee', 'Trump - Outside Groups', 'Trump - Combined', 'Biden - Campaign Committee', 'Biden - Outside Groups', 'Biden - Combined']
stacked_df.set_index('Candidate', inplace=True)
stacked_df = stacked_df.transpose()
stacked_df = stacked_df.reset_index().rename(columns={'index': 'Sector'})

# create function to compare and add asterisk
def add_asterisk(row):
    # compare columns 2 and 5
    if row[1] > row[4]:
        row[1] = str(row[1]) + '*'
    else:
        row[4] = str(row[4]) + '*'

    # compare columns 3 and 6
    if row[2] > row[5]:
        row[2] = str(row[2]) + '*'
    else:
        row[5] = str(row[5]) + '*'

    # compare columns 4 and 7
    if row[3] > row[6]:
        row[3] = str(row[3]) + '*'
    else:
        row[6] = str(row[6]) + '*'

    return row

# apply function to each row of dataframe
stacked_df = stacked_df.apply(add_asterisk, axis=1)

# create a figure and axis
fig, ax = plt.subplots()

# hide axis
ax.axis('off')

# create the table using the data in the dataframe
table = ax.table(cellText=stacked_df.values, colLabels=stacked_df.columns, loc='center', cellLoc='center')

# set the font size of the cells and header
table.set_fontsize(14)
table.auto_set_column_width(col=list(range(len(stacked_df.columns))))

# Set the height of the cells
table.scale(1, 2)

# Set the title of the table
title = 'Funding by Industry, Candidate, and Channel (Millions of $)'
plt.text(0, 0.08, title, ha='center', va='top', fontsize=16)

# Hide the axis and turn off the axis lines and ticks
ax.axis('off')
ax.axis('tight')

# add note below the plot
note = "*:  Via this channel, the amount of funding gathered for this candidate exceeds the funding collected for the other candidate."
plt.text(-0.2, -0.075, note, ha='left', va='bottom', fontsize=12)

# Show the plot
plt.show()

To summarize the table, Trump received more funds than Biden on these sectors: Agribusiness, Construction, Energy/Natrual Resources, Health, Transportation, Misc Business, and Other; while Biden received more funds than Trump on these sectors: Communications/Electronics, Defense, Finance/Insurance/Real Estate, Lawyers & Lobbyists, Labor, and Ideology/Single-Issue.

Trump's focus on deregulation and reducing environmental protections may have made him more attractive to industries such as Agribusiness, Construction, and Energy/Natural Resources, which often face regulatory hurdles that can be costly and time-consuming. On the other hand, Biden's policies on climate change, healthcare, and workers' rights may have resonated more with the Communications/Electronics, Defense, and Labor sectors.

Furthermore, the support of certain industries for a particular candidate can also be influenced by the interests of the businesses or individuals within those industries. For instance, the Finance/Insurance/Real Estate sector may have supported Biden due to his policies on taxes and financial regulation, while the Lawyers & Lobbyists sector may have supported him due to his support for civil rights and social justice issues.

The fact that Trump received more funds than Biden in the Transportation and Health sectors could also be attributed to his administration's handling of the COVID-19 pandemic, which heavily impacted these industries. Trump's focus on reopening the economy and his skepticism towards public health measures may have resonated with certain businesses within these sectors.

Overall, the differences in funding received by Trump and Biden in different sectors highlight the complex interplay between policy positions, industry interests, and political strategy in the 2020 US Presidential election. It also suggests that different groups of voters may have been more attracted to either Trump's or Biden's policies, depending on their respective industries or personal interests.

In [ ]:
# PLOT
# define the column groups
groups = ['Agribusiness', 'Communications/Electronics', 'Construction', 'Defense', 'Energy/Natural Resources',
          'Finance/Insurance/Real Estate', 'Health', 'Lawyers & Lobbyists', 'Transportation', 'Misc Business',
          'Labor', 'Ideology/Single-Issue', 'Other']

# define the two candidates
candidates = ['Trump, Donald (R)', 'Biden, Joe (D)']

# define the x-axis positions for each group of bars
x_positions = np.arange(len(groups)) * 3

# define the bar width
bar_width = 0.8

# create the figure and axis objects
fig, ax = plt.subplots(figsize=(12, 6))

# define the labels for the legend
labels = ['Trump - Campaign Committee', 'Trump - Outside Groups', 'Biden - Campaign Committee', 'Biden - Outside Groups']

# loop over each group of columns
for i in range(1, df1.shape[1]):

    # get the data for this group from the two dataframes
    data1 = np.array([df1.loc[df1['Candidate'] == 'Trump, Donald (R)', df_funds[1].columns[i]].values[0],
                      df1.loc[df1['Candidate'] == 'Biden, Joe (D)', df_funds[1].columns[i]].values[0]])
    data2 = np.array([df2.loc[df2['Candidate'] == 'Trump, Donald (R)', df_funds[2].columns[i]].values[0],
                      df2.loc[df2['Candidate'] == 'Biden, Joe (D)', df_funds[2].columns[i]].values[0]])

    # calculate the x-axis positions for the two bars in this group
    x_pos_left = x_positions[i - 1] - bar_width / 2
    x_pos_right = x_positions[i - 1] + bar_width / 2

    # plot the bars for the two candidates, stacked on top of each other
    ax.bar(x_pos_left, data1[0], width=bar_width, label=labels[0], color='darkred')
    ax.bar(x_pos_left, data1[1], width=bar_width, bottom=data1[0], color='lightcoral')
    ax.bar(x_pos_right, data2[0], width=bar_width, label=labels[2], color='navy')
    ax.bar(x_pos_right, data2[1], width=bar_width, bottom=data2[0], color='lightskyblue')

# set the x-axis tick labels to be the group names
ax.set_xticks(x_positions)
ax.set_xticklabels(groups, rotation=45, ha = 'right')

# set the legend parameters
ax.legend(labels=labels, loc='upper left', title=None)

# set the axis labels and title
ax.set_xlabel('Industry')
ax.set_ylabel('Total Funding (Millions of $)')
ax.set_title('Funding by Industry and Candidate')

# show the plot
plt.show()

From the above bar chart, for both Trump and Biden, the funding from the outside groups is almost equal to if not larger than the funding from the campaign committee. The fact is significant for several reasons, particularly in relation to the 2020 US Presidential election.

Firstly, it suggests that the official campaign committees of both Trump and Biden may have been limited in their ability to raise funds, or may have chosen to focus their efforts elsewhere, such as on grassroots organizing or digital advertising. This could reflect a shift in campaign strategy towards more decentralized, bottom-up approaches to fundraising and mobilization.

Secondly, the fact that the outside groups' funding is almost equal to or larger than the campaign committee funding for both candidates suggests that the 2020 US Presidential election was fiercely competitive and hotly contested. Both campaigns were likely aware of the importance of funding in securing victory, and may have gone to great lengths to secure support from outside groups, particularly those with deep pockets and significant influence.

Overall, the significant role played by outside groups in the 2020 US Presidential election highlights the importance of money and resources in modern political campaigns.

Another graph: pie chart of Trump and Biden

In [ ]:
# Define the desired column order
new_order = ['Candidate',
             'Agribusiness', 'Communications/Electronics', 'Construction',
             'Labor', 'Finance/Insurance/Real Estate', 'Health',
             'Energy/Natural Resources', 'Lawyers & Lobbyists', 'Misc Business',
             'Defense', 'Ideology/Single-Issue', 'Transportation',
             'Other']

# Use reindex to reorder the columns
df00 = df0.reindex(columns=new_order)

# Filter data for Trump and Biden
trump_data = df00[df00['Candidate'] == 'Trump, Donald (R)'].iloc[:, 1:]
biden_data = df00[df00['Candidate'] == 'Biden, Joe (D)'].iloc[:, 1:]

# Sum contributions across sectors
trump_total = trump_data.sum(axis=0)
biden_total = biden_data.sum(axis=0)

# Set up colors for the pie chart
colors = ['#ffd700', '#7fffd4', '#cd5c5c',
          '#ff69b4', '#20b2aa', '#ff4500',
          '#00ced1', '#9370db', '#696969',
        '#8b0000', '#ffff00', '#808080',
          '#8fbc8f']

# Set up labels for the pie chart
labels_trump = trump_data.columns
labels_biden = biden_data.columns

# Set up explode parameter for the pie chart
explode_trump = [0.05] * len(labels_trump)
explode_biden = [0.05] * len(labels_biden)

# Plot the pie chart
fig, ax = plt.subplots(figsize=(6, 6))
ax.pie(trump_total, labels=labels_trump, colors=colors, explode=explode_trump, autopct='%1.1f%%', textprops={'fontsize': 10})
ax.set_title("Trump's Donors by Sector")

fig, ax = plt.subplots(figsize=(6, 6))
ax.pie(biden_total, labels=labels_biden, colors=colors, explode=explode_biden, autopct='%1.1f%%', textprops={'fontsize': 10})
ax.set_title("Biden's Donors by Sector")

plt.show()


In the above two pie charts, it is clear that for both Trump and Biden, the Finance/Insurance/Real Estate sector is contributing substantially to the fundraising. One reason could be that this sector is intrinsically holding higher margins in their business, and the other reason could be that its business environment is heavily depending on the government's policy, so they have larger incentives to influence the result of the election.

For Trump, the Other sector stands out from all and ranked top for the highest contribution. It would be attributed to Trump's business career before he ran for the US President. As a billionaire real estate developer, he made use of the Trump Organization to provide him huge financial support for his campaign. And this contribution is categorized in the Other sector.

For Biden, the portion of Ideology/Single-Issue sector is significantly higher than its portion in Trump's funds pool. The Ideology/Single-Issue sector includes organizations that focus on a single-issue area or are heavily partisan, such as those related to abortion, gun control, healthcare, environment, and foreign policy. Therefore, the reason could be that some certain groups - which were likely taking a more liberal stance on their issue - were severely unsatisfied with Trump administration from 2016 to 2020, so that they choose to firmly support Trump's opposition, Biden, by donating to his campaign financially.

## Adding a New Dataset

### First Dataset
New Dataset: https://www.kaggle.com/datasets/noriuk/us-education-datasets-unification-project?resource=download&select=states_all.csv

Recall that the one of purposes of this paper was to study what kind of citizen would vote for Trump in the 2020 Election. From the previous discussion, we learned that Trump was more favored by people holding conservative views, who may potentially have lower educational attainments. This dataset contains the percentage of population over 25 years old who have obtained Bachelor's degree or higher, which can be a good approximation for the educational attainment level of each state when the 2020 election happened.

In [ ]:
df_higher_edu = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Other/2020_education.csv')
df_higher_edu["Bachelor's degree or higher"] = df_higher_edu["Bachelor's degree or higher"].replace('%', '', regex=True).astype(float)
df_higher_edu.rename(columns={"Bachelor's degree or higher":"Bachelor's degree or higher (%)"}, inplace = True)

df_higher_edu.head()

Scatter plot of Educational attainment and vote share of Trump

In [ ]:
compare_edu_vote = pd.merge(df_higher_edu, df_trump_vote_sum, on='NAME', how='left')
compare_edu_vote.head()

In [ ]:
fig = plt.figure(figsize=(15, 15))
plt.scatter(compare_edu_vote['average_score'], compare_edu_vote['ratio'] * 100)

for i, label in enumerate(compare_edu_vote['NAME']):
    plt.annotate(label, (
    compare_edu_vote['average_score'][i], compare_edu_vote['ratio'][i] * 100))

plt.xlabel('Educational Average Score')
plt.ylabel('% Vote for Trump')
plt.ylim([0, 100])
plt.show()

### Second Dataset
New Dataset: https://www.kaggle.com/datasets/goldenoakresearch/us-household-income-stats-geo-locations

In [ ]:
df_income = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Other/income.csv')
df_income.rename(columns={"State_Name": "NAME"}, inplace=True)
df_income.head()

In [ ]:
compare_income_vote = pd.merge(df_income, df_trump_vote_sum, on='NAME', how='left')
compare_income_vote.dropna(inplace=True)
compare_income_vote.head()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
sns.scatterplot(data=compare_income_vote, x='Average of Mean', y= compare_income_vote['ratio']*100, ax=ax1)
sns.scatterplot(data=compare_income_vote, x='Average of Median', y= compare_income_vote['ratio']*100, ax=ax2)
sns.scatterplot(data=compare_income_vote, x='Average of Stdev', y= compare_income_vote['ratio']*100, ax=ax3)
ax1.set_ylabel("% Vote for Trump", fontsize=14)
ax1.set_xlabel("Mean of Income", fontsize=14)
ax2.set_xlabel("Median of Income", fontsize=14)
ax3.set_xlabel("Standard Deviation of Income", fontsize=14)
ax1.set_ylim([0, 100])
fig.suptitle("Income Features VS % Vote for Trump on State Level")
plt.show()


# OLS Regression

1. VEP Turnout ~ log(population), log(density), Median age of the population

In [ ]:
df_voter = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Kaggle/2020 November General Election - Turnout Rates.csv')
df_voter.head()

In [ ]:
df_age = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Other/age.csv')
df_age.head()

In [ ]:
df_voter_turnout = pd.merge(df_voter2, df_income[['NAME', 'Average of Mean']], on='NAME', how='left')
df_voter_turnout['VEP Turnout Rate (%)'] = df_voter_turnout[
    'VEP Turnout Rate (%)'].replace('%', '', regex=True).astype(float)
df_voter_turnout['Average of Mean'] = df_voter_turnout['Average of Mean'] / 1000
df_voter_turnout.rename(
    columns={"Average of Mean": "Avg Annual Income (USD1,000)"}, inplace=True)
df_voter_turnout = df_voter_turnout.iloc[1:]
df_voter_turnout = pd.merge(df_voter_turnout, df_age, on='NAME', how='left')
df_voter_turnout.head()

In [ ]:
df_voter_turnout = pd.DataFrame()
df_voter_turnout['NAME'] = df_voter['State']
df_voter_turnout['VEP Turnout Rate (%)'] = df_voter['VEP Turnout Rate']
df_voter_turnout['Log of Voting-Eligible Population (VEP)'] = \
    np.log10(df_voter['Voting-Eligible Population (VEP)'].replace(',', '', regex=True).astype(int))
df_voter_turnout = pd.merge(df_voter_turnout, df_income[['NAME', 'Average of Mean']], on='NAME', how='left')
df_voter_turnout['VEP Turnout Rate (%)'] = df_voter_turnout['VEP Turnout Rate (%)'].replace('%', '', regex=True).astype(float)
df_voter_turnout['Average of Mean'] = df_voter_turnout['Average of Mean']/1000
df_voter_turnout.rename(columns={"Average of Mean": "Avg Annual Income (USD1,000)"}, inplace = True)
df_voter_turnout = df_voter_turnout.iloc[1:]
df_voter_turnout = pd.merge(df_voter_turnout, df_age, on='NAME', how='left')
df_voter_turnout.head()

Regression for VEP Turnout:

In [ ]:
df_voter_turnout['const'] = 1
reg1 = sm.OLS(endog = df_voter_turnout['VEP Turnout Rate (%)'], exog = df_voter_turnout[['const', 'Median Age (years)']], missing = 'drop')
reg2 = sm.OLS(endog = df_voter_turnout['VEP Turnout Rate (%)'], exog = df_voter_turnout[['const', 'Median Age (years)', 'Avg Annual Income (USD1,000)']], missing = 'drop')
reg3 = sm.OLS(endog = df_voter_turnout['VEP Turnout Rate (%)'], exog = df_voter_turnout[['const', 'Median Age (years)', 'Avg Annual Income (USD1,000)', 'Log of Voting-Eligible Population (VEP)']], missing = 'drop')
result1 = reg1.fit()
result2 = reg2.fit()
result3 = reg3.fit()
stargazer = Stargazer([result1, result2, result3])
stargazer.custom_columns(['Model 1', 'Model 2', 'Model 3'], [1,1,1])
stargazer.title("Regression Results")
HTML(stargazer.render_html())

In [ ]:
df_1 = df_voter_turnout.copy()

# Dropping NA's is required to use numpy's polyfit
df_2 = df_1.dropna(subset=['VEP Turnout Rate (%)', 'Median Age (years)'])

X = df_2['Median Age (years)']
y = df_2['VEP Turnout Rate (%)']
labels = df_2['NAME']

# Replace markers with country labels
fig, ax = plt.subplots()
ax.scatter(X, y, marker='o')

# Fit a linear trend line
ax.plot(np.unique(X),
         np.poly1d(np.polyfit(X, y, 1))(np.unique(X)),
         color='black')

ax.set_xlabel('Median Age (years)')
ax.set_ylabel('VEP Turnout Rate (%)')
ax.set_title('Figure 2: OLS relationship between Median Age and VEP Turnout Rate')
plt.show()

2. Vote Share ~ income standard deviation, area of land per capita, unemployment rate

$$\text{vote share (\%)} = 100*\frac{\text{total amount of trump vote in a state} / \text{total amount of vote in a state}}$$

In [ ]:
df_unemployment = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Other/unemployment_rate.csv')
df_unemployment['Unemployment Rate'] = df_unemployment['Unemployment Rate'].replace('%', '', regex=True).astype(float)
df_unemployment = df_unemployment.rename(columns={'Unemployment Rate': 'Unemployment Rate (%)', 'State':'NAME'})
df_unemployment.head()

In [ ]:
df_vote_share = pd.merge(compare_income_vote, df_higher_edu, on='NAME', how='left')
df_vote_share = pd.merge(df_vote_share, df_unemployment, on='NAME', how='left')
df_vote_share = df_vote_share.loc[:, ['NAME', 'ratio', 'Average of Stdev', "Bachelor's degree or higher (%)", 'Unemployment Rate (%)']]
df_vote_share['Vote Share of Trump (%)'] = df_vote_share['ratio']*100
#df_vote_share['Land Per Capita'] = df_vote_share['Land Per Capita']/10000
df_vote_share['Average of Stdev'] = df_vote_share['Average of Stdev']/10000
df_vote_share.drop(['ratio'],axis = 1, inplace = True)
df_vote_share.rename(columns={"Average of Stdev": "Annual Income Standard Deviation (USD10,000)"}, inplace = True)

df_vote_share.head()

Regression for Vote Share:

In [ ]:
df_vote_share['const'] = 1
reg1 = sm.OLS(endog = df_vote_share['Vote Share of Trump (%)'], exog = df_vote_share[['const', 'Annual Income Standard Deviation (USD10,000)']], missing = 'drop')
reg2 = sm.OLS(endog = df_vote_share['Vote Share of Trump (%)'], exog = df_vote_share[['const', 'Annual Income Standard Deviation (USD10,000)', 'Unemployment Rate (%)']], missing = 'drop')
reg3 = sm.OLS(endog = df_vote_share['Vote Share of Trump (%)'], exog = df_vote_share[['const', 'Annual Income Standard Deviation (USD10,000)', 'Unemployment Rate (%)', "Bachelor's degree or higher (%)"]], missing = 'drop')
result1 = reg1.fit()
result2 = reg2.fit()
result3 = reg3.fit()
stargazer = Stargazer([result1, result2, result3])
stargazer.custom_columns(['Model 1', 'Model 2', 'Model 3'], [1,1,1])
stargazer.title("Regression Results")
HTML(stargazer.render_html())

In [ ]:
df_1 = df_vote_share.copy()

# Dropping NA's is required to use numpy's polyfit
df_2 = df_1.dropna(subset=['Vote Share of Trump (%)', "Bachelor's degree or higher (%)"])

X = df_2["Bachelor's degree or higher (%)"]
y = df_2['Vote Share of Trump (%)']
labels = df_2['NAME']

# Replace markers with country labels
fig, ax = plt.subplots()
ax.scatter(X, y, marker='o')

# Fit a linear trend line
ax.plot(np.unique(X),
         np.poly1d(np.polyfit(X, y, 1))(np.unique(X)),
         color='black')

ax.set_xlabel("Bachelor's degree or higher (%)")
ax.set_ylabel('Vote Share of Trump (%)')
ax.set_title("Figure 2: OLS relationship between Population with Bachelor's degree or higher (%) and Vote Share of Trump")
plt.show()

In [ ]:
df_1 = df_vote_share.copy()

# Dropping NA's is required to use numpy's polyfit
df_2 = df_1.dropna(subset=['Vote Share of Trump (%)', 'Annual Income Standard Deviation (USD10,000)'])

X = df_2['Annual Income Standard Deviation (USD10,000)']
y = df_2['Vote Share of Trump (%)']
labels = df_2['NAME']

# Replace markers with country labels
fig, ax = plt.subplots()
ax.scatter(X, y, marker='o')

# Fit a linear trend line
ax.plot(np.unique(X),
         np.poly1d(np.polyfit(X, y, 1))(np.unique(X)),
         color='black')

ax.set_xlabel('Annual Income Standard Deviation (USD10,000)')
ax.set_ylabel('Vote Share of Trump (%)')
ax.set_title('Figure 2: OLS relationship between Annual Income Standard Deviation and Vote Share of Trump')
plt.show()

Political Polarization ~ population of one race/total population, proportion of educational attainment - bachelor's degree or higher (age over 25), # of campaigns hosted

Formula for polarization indicator:
|total amount of trump votes in a state - total amount of biden vote in a state| / all other votes in a state


In [ ]:
# create a new column to group candidates as "Donald Trump", "Joe Biden", or "Others"
df['Candidate_Group'] = df['candidate'].apply(lambda x: x if x in ['Donald Trump', 'Joe Biden'] else 'Others')

# group the data by state and candidate group and sum the total_votes column
grouped_df = df.groupby(['state', 'Candidate_Group'])['total_votes'].sum().reset_index()

# pivot the table on the Candidate_Group column
pivoted_df = grouped_df.pivot_table(index='state', columns='Candidate_Group', values='total_votes', aggfunc='sum').reset_index()

# rename the columns to the desired names
pivoted_df.columns = ['NAME', 'Donald Trump', 'Joe Biden', 'Others']

pivoted_df['Polarization Indicator'] = np.abs(pivoted_df['Donald Trump']-pivoted_df['Joe Biden'])/pivoted_df['Others']

pivoted_df.head()

In [ ]:
df_pop = pd.read_csv('/Users/jiaxincui/Documents/GitHub/git-exercise-AllysonCui/Data/Other/DECENNIALPL2020.P2-2023-03-31T211327.csv')
df_pop1 = df_pop.loc[df_pop['Label (Grouping)'].isin(['Total:', '        Population of one race:'])]
df_pop1 = df_pop1.T
df_pop1.columns = df_pop1.iloc[0]
df_pop1 = df_pop1.drop(df_pop1.index[0])
df_pop1 = df_pop1.reset_index().rename(columns={'index': 'NAME'})
df_pop1.rename(columns={'Total:':'Total',
                        "        Population of one race:": "Population of one race",
                        'Label (Grouping)':'NAME'}, inplace = True)
df_pop1['Total'] = df_pop1['Total'].replace(',', '', regex=True).astype(int)
df_pop1['Population of one race'] = df_pop1['Population of one race'].replace(',', '', regex=True).astype(int)
df_pop1.head()

In [ ]:
df_polar = pd.DataFrame()
df_polar['NAME'] = pivoted_df['NAME']
df_polar['Polarization Indicator'] = pivoted_df['Polarization Indicator']

df_race = df_pop1.copy()
df_race['Proportion of Single Race Population'] = df_race['Population of one race'] / df_race['Total']
df_polar = pd.merge(df_polar, df_race[['NAME', 'Proportion of Single Race Population']], on='NAME', how='left')
df_polar['Proportion of Single Race Population'] = df_polar['Proportion of Single Race Population']*100

df_polar = pd.merge(df_polar, df_higher_edu, on='NAME', how='left')

df_campaign = Rallies_2020.copy()
df_campaign.rename(columns={"State": "STUSPS"}, inplace = True)
df_campaign = pd.merge(df_campaign, gpd_state4[['STUSPS', 'NAME']], on='STUSPS', how='left')

name_counts = df_campaign['NAME'].value_counts()
df_campaign_num = pd.DataFrame({'NAME': name_counts.index, 'count': name_counts.values})

df_polar = pd.merge(df_polar, df_campaign_num[['NAME', 'count']], on='NAME', how='left')
df_polar['count'] = df_polar['count'].fillna(0)
df_polar.rename(columns={"count": "# of 2020 Presidential Campaigns", 'Proportion of Single Race Population':'Proportion of Single Race Population (%)'}, inplace = True)

df_polar = pd.merge(df_polar, df_voter_turnout[['NAME', 'VEP Turnout Rate (%)']], on='NAME', how='left')

df_polar.head()

Regression for Political Polarization:

In [ ]:
df_polar['const'] = 1
var2 = 'VEP Turnout Rate (%)'
var1 = "# of 2020 Presidential Campaigns"
var3 = "Bachelor's degree or higher (%)"
var4 = 'Proportion of Single Race Population (%)'
reg1 = sm.OLS(endog = df_polar['Polarization Indicator'],
              exog = df_polar[['const', var1]], missing = 'drop')
reg2 = sm.OLS(endog = df_polar['Polarization Indicator'],
              exog = df_polar[['const', var1, var2]], missing = 'drop')
reg3 = sm.OLS(endog = df_polar['Polarization Indicator'],
              exog = df_polar[['const', var1, var2, var3]], missing = 'drop')
reg4 = sm.OLS(endog = df_polar['Polarization Indicator'],
              exog = df_polar[['const', var1, var2, var3, var4]], missing = 'drop')
result1 = reg1.fit()
result2 = reg2.fit()
result3 = reg3.fit()
result4 = reg4.fit()
stargazer = Stargazer([result1, result2, result3, result4])
stargazer.custom_columns(['Model 1', 'Model 2', 'Model 3', 'Model 4'], [1,1,1,1])
stargazer.title("Regression Results")
HTML(stargazer.render_html())

df_test = pd.merge(df_polar, df_vote_share, on = "NAME", how='left')
df_test = pd.merge(df_test, df_voter_turnout, on="NAME", how='left')
df_test = pd.merge(df_test, df_state_status, on="NAME", how='left')

df_test.drop(['const_x', 'const_y'], axis = 1, inplace = True)
df_test.head()

df_test['const'] = 1

reg1 = sm.OLS(endog = df_test['Relative Winning Ratio (%)'],
              exog = df_test[['const', "Proportion of White Alone Population"]], missing = 'drop')
reg2 = sm.OLS(endog = df_test['Relative Winning Ratio (%)'],
              exog = df_test[['const', "Proportion of Single Race Population (%)"]], missing = 'drop')
reg3 = sm.OLS(endog = df_test['Relative Winning Ratio (%)'],
              exog = df_test[['const', "# of 2020 Presidential Campaigns"]], missing = 'drop')
reg4 = sm.OLS(endog = df_test['Relative Winning Ratio (%)'],
              exog = df_test[['const', "Annual Income Standard Deviation (USD10,000)"]], missing = 'drop')
reg5 = sm.OLS(endog = df_test['Relative Winning Ratio (%)'],
              exog = df_test[['const', "Unemployment Rate (%)"]], missing = 'drop')
reg6 = sm.OLS(endog = df_test['Relative Winning Ratio (%)'],
              exog = df_test[['const', "Median Age (years)"]], missing = 'drop')
result1 = reg1.fit()
result2 = reg2.fit()
result3 = reg3.fit()
result4 = reg4.fit()
result5 = reg5.fit()
result6 = reg6.fit()
stargazer = Stargazer([result1, result2, result3, result4, result5, result6])
stargazer.custom_columns(['Model 1', 'Model 2', 'Model 3', 'Model 4', 'Model 5', 'Model 6' ], [1,1,1,1,1,1])
stargazer.title("Regression Results")
HTML(stargazer.render_html())

In [ ]:
df_1 = df_polar.copy()

# Dropping NA's is required to use numpy's polyfit
df_2 = df_1.dropna(subset=['Polarization Rate (%)', "Bachelor's degree or higher (%)"])

X = df_2["Bachelor's degree or higher (%)"]
y = df_2['Polarization Rate (%)']
labels = df_2['NAME']

# Replace markers with country labels
fig, ax = plt.subplots()
ax.scatter(X, y, marker='o')

# Fit a linear trend line
ax.plot(np.unique(X),
         np.poly1d(np.polyfit(X, y, 1))(np.unique(X)),
         color='black')

ax.set_xlabel("Population with Bachelor's degree or higher (%)")
ax.set_ylabel('Polarization Rate (%)')
ax.set_title("Figure 2: OLS relationship between Polarization Rate and Population with Bachelor's degree or higher")
plt.show()

State-Level Voting Winning Status ~ median of % Trump vote share in counties, log of median of county population, # of counties,

In [ ]:
# group the dataframe by state and compute median values
state_grouped = df_county_pop.groupby('state').agg({'trump_vote_share': 'median', 'population': 'max'})

# reset the index to make state a regular column
state_grouped = state_grouped.reset_index()

# rename the columns
state_grouped = state_grouped.rename(columns={'state': 'NAME', 'trump_vote_share': 'Median County Vote Share of Trump (%)', 'population': 'Highest County Population'})

# create the new dataframe
#df_state_pop = pd.DataFrame()
df_state_pop = state_grouped[['NAME', 'Median County Vote Share of Trump (%)', 'Highest County Population']]

df_state_pop['Median County Vote Share of Trump (%)'] = df_state_pop['Median County Vote Share of Trump (%)']*100

df_counties = df_county.groupby('state')['county'].nunique().reset_index()
df_counties.columns = ['NAME', '# of Counties']

In [ ]:
df_state_status = pd.DataFrame()
vmin = gpd_state4['total_votes'].min()
vmax = gpd_state4['total_votes'].max()
df_state_status['NAME'] = gpd_state4['NAME']
df_state_status['State-Level Trump Vote Contribution (%)'] = (gpd_state4['total_votes']-vmin)/(vmax-vmin)*100
names_list = list(gpd_state4.loc[gpd_state4['party'] != 'REP']['NAME'])
#Need to double check
df_state_status.loc[df_state_status['NAME'].isin(names_list), 'State-Level Trump Vote Contribution (%)'] = 0
df_state_status = pd.merge(df_state_status, df_state_pop, on='NAME', how='left')
df_state_status = pd.merge(df_state_status, df_counties, on='NAME', how='left')
df_state_status = pd.merge(df_state_status, df_polar[['NAME', "# of 2020 Presidential Campaigns"]], on='NAME', how='left')

df_state_status.head()

In [ ]:
df_state_status['const'] = 1
reg1 = sm.OLS(endog = df_state_status['State-Level Trump Vote Contribution (%)'],
              exog = df_state_status[['const', 'Highest County Population']], missing = 'drop')
reg2 = sm.OLS(endog = df_state_status['State-Level Trump Vote Contribution (%)'],
              exog = df_state_status[['const', 'Highest County Population', '# of 2020 Presidential Campaigns']], missing = 'drop')
reg3 = sm.OLS(endog = df_state_status['State-Level Trump Vote Contribution (%)'],
              exog = df_state_status[['const', 'Highest County Population', '# of 2020 Presidential Campaigns', 'Median County Vote Share of Trump (%)']], missing = 'drop')
reg4 = sm.OLS(endog = df_state_status['State-Level Trump Vote Contribution (%)'],
              exog = df_state_status[['const', 'Highest County Population', '# of 2020 Presidential Campaigns', 'Median County Vote Share of Trump (%)', "# of Counties"]], missing = 'drop')
result1 = reg1.fit()
result2 = reg2.fit()
result3 = reg3.fit()
result4 = reg4.fit()
stargazer = Stargazer([result1, result2, result3, result4])
stargazer.custom_columns(['Model 1', 'Model 2', 'Model 3', 'Model 4'], [1,1,1,1])
stargazer.title("Regression Results")
HTML(stargazer.render_html())

In [ ]:
df_1 = df_state_status.copy()

# Dropping NA's is required to use numpy's polyfit
df_2 = df_1.dropna(subset=['State-Level Trump Vote Contribution (%)', 'Highest County Population'])

X = df_2['Highest County Population']
y = df_2['State-Level Trump Vote Contribution (%)']
labels = df_2['NAME']

# Replace markers with country labels
fig, ax = plt.subplots()
ax.scatter(X, y, marker='o')

# Fit a linear trend line
ax.plot(np.unique(X),
         np.poly1d(np.polyfit(X, y, 1))(np.unique(X)),
         color='black')

#ax.set_xlim([3.3,10.5])
#ax.set_ylim([0,100])
ax.set_xlabel('Highest County Population')
ax.set_ylabel("State-Level Trump Vote Contribution (%)")
ax.set_title("Figure 2: OLS relationship between Highest County Population and State-Level Trump Vote Contribution")
plt.show()

# Machine Learning

In [ ]:
from sklearn import (
    linear_model, metrics, pipeline, model_selection
)
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,confusion_matrix, classification_report
from sklearn import tree
import plotly.graph_objects as go
import qeds

In [ ]:
X = df_2.drop(["NAME"], axis=1).copy()
# convert everything to be a float for later on
for col in list(X):
    X[col] = X[col].astype(float)
X.head()

In [ ]:
lr_model = linear_model.LinearRegression()
lr_model.fit(X, y)

In [ ]:
y_pred_linear = lr_model.predict(X)
#from sklearn import metrics
full_mse = metrics.mean_squared_error(y, y_pred_linear)
print('Mean Squared Error:', full_mse)

Regression Tree

In [ ]:
sqft_tree = tree.DecisionTreeRegressor(max_depth=3).fit(X,y)

In [ ]:
# use the fitted tree to predict
y_pred_tree = sqft_tree.predict(X)

# find the error of prediction (MSE)
print('Mean Squared Error:', metrics.mean_squared_error(y, y_pred_tree))

In [ ]:
sqrf_fig = plt.figure(figsize=(20,10))
sqrf_fig = tree.plot_tree(sqft_tree, feature_names=X.columns, filled=True, fontsize = 12)

In [ ]:
Importance = pd.DataFrame({'Importance':regr2.feature_importances_*100}, index=X.columns)
Importance.sort_values('Importance', axis=0, ascending=True).plot(kind='barh', color='r', )
plt.xlabel('Variable Importance')
plt.gca().legend_ = None

RandomForest

In [ ]:
regr2 = RandomForestRegressor(max_features=2, random_state=1)
regr2.fit(X, y)
pred = regr2.predict(X)
mean_squared_error(y, pred)

# Regression Tree for Random Forest
n = 1000
Xsim = np.random.rand(n,2)
def Ey_x(x):
    return 1/3*(np.sin(5*x[0])*np.sqrt(x[1])*np.exp(-(x[1]-0.5)**2))
#np.apply_along_axis applys a function (Ey_x here) to 1-D slices along the given axis which is Xsim here
ysim = np.apply_along_axis(Ey_x, 1, Xsim) + np.random.randn(n)*0.1

qeds.themes.mpl_style();
plotly_template = qeds.themes.plotly_template()
colors = qeds.themes.COLOR_CYCLE

#defining a 3D scatter plot to show the data
def surface_scatter_plot(X,y,f, xlo=0., xhi=1., ngrid=50,
                         width=860, height=700, f0=Ey_x, show_f0=False):
    scatter = go.Scatter3d(x=X[:,0],y=X[:,1],z=y,
                           mode='markers',
                           marker=dict(size=2, opacity=0.3)
    )
    xgrid = np.linspace(xlo,xhi,ngrid)
    ey = np.zeros((len(xgrid),len(xgrid)))
    ey0 = np.zeros((len(xgrid),len(xgrid)))
    colorscale = [[0, colors[0]], [1, colors[2]]]
    for i in range(len(xgrid)):
        for j in range(len(xgrid)):
            ey[j,i] = f([xgrid[i],xgrid[j]])
            ey0[j,i]= f0([xgrid[i],xgrid[j]])
    surface = go.Surface(x=xgrid, y=xgrid, z=ey, colorscale=colorscale, opacity=1.0)
    if (show_f0):
        surface0 = go.Surface(x=xgrid, y=xgrid, z=ey0, opacity=0.8, colorscale=colorscale)
        layers = [scatter, surface, surface0]
    else:
        layers = [scatter, surface]
    fig = go.FigureWidget(
        data=layers,
        layout = go.Layout(
            autosize=True,
            scene=dict(
                xaxis_title='X1',
                yaxis_title='X2',
                zaxis_title='Y'
            ),
            width=width,
            height=height,
            template=plotly_template,
        )
    )
    return fig

fig = surface_scatter_plot(Xsim, ysim, Ey_x)
fig

fitted_tree = tree.DecisionTreeRegressor(max_depth=3).fit(Xsim,ysim)
fig = plt.figure(figsize=(25,20))
fig = tree.plot_tree(fitted_tree, feature_names=["var1", "var2"], filled=True)


## Conclusion
In conclusion, the results of the four different electoral systems applied to the same election data reveal the significance of the electoral process in determining the outcome of an election. The outcome can vary greatly depending on the specific system being used. In this case, Joe Biden was declared the winner under Electoral System I, where each county was allotted the same number of votes as its number of voters and all votes were cast for the candidate who won the majority of votes in that county. On the other hand, Donald Trump was declared the winner in the remaining three systems - Electoral System II, III, and IV.

In [ ]:
result_4 = [['I', 'Joe Biden'],
            ['II', 'Donald Trump'],
            ['III', 'Donald Trump'],
            ['IV', 'Donald Trump']]
pd.DataFrame(result_4, columns = ['Electoral System', 'Winner'])

 This highlights the importance of choosing an appropriate electoral system that aligns with the values and goals of a given society. It also demonstrates how different electoral systems can have a significant impact on the representation of different regions, communities, and individuals. The analysis shows that the design of an electoral system can greatly influence the outcome of an election and should be carefully considered.

The differences in funding received by Trump and Biden in different sectors also suggest the complex interplay between policy positions, industry interests, and political strategy in the 2020 US Presidential election. It underscores the importance of money and resources in modern political campaigns and reflects the changing dynamics of American politics.

This research paper also highlights the need for ongoing research into the role of outside groups in political campaigns, particularly in the context of the increasing influence of money and resources.